##### Copyright 2021 Google LLC. Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License

# This colab retrieves urls of (Cloud Optimized) GeoTIFFs for a given region from the Open Buildings 2.5D Temporal Dataset.





In [ ]:
#@title Imports (need to be run only once)

!sudo apt-get install swig
!pip install rasterio s2geometry pygeos geopandas tqdm

import functools
import glob
import gzip
import json
import multiprocessing
from multiprocessing.pool import ThreadPool
import os
import shutil
import tempfile
from typing import Optional, Tuple, Iterable, Callable, Any

import geopandas as gpd
from google.auth import credentials
from google.cloud import storage
from IPython import display
import pandas as pd
import pyproj
import rasterio
from rasterio.transform import Affine
import s2geometry as s2
import shapely
from shapely.geometry import Polygon, box
import tqdm.notebook
from shapely.ops import transform

In [ ]:
# @title Prepare urls of GeoTIFFs of the given region

# @markdown First, select a region from either the [Natural Earth low res](https://www.naturalearthdata.com/downloads/110m-cultural-vectors/110m-admin-0-countries/) (fastest), [Natural Earth high res](https://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-admin-0-countries/) or [World Bank high res](https://datacatalog.worldbank.org/dataset/world-bank-official-boundaries) shapefiles:
region_border_source = "Natural Earth (High Res 10m)"  # @param ["Natural Earth (Low Res 110m)", "Natural Earth (High Res 10m)", "World Bank (High Res 10m)"]
region = "CRI (Costa Rica)"  # @param ["", "ABW (Aruba)", "AGO (Angola)", "AIA (Anguilla)", "ARG (Argentina)", "ATG (Antigua and Barbuda)", "BDI (Burundi)", "BEN (Benin)", "BFA (Burkina Faso)", "BGD (Bangladesh)", "BHS (The Bahamas)", "BLM (Saint Barthelemy)", "BLZ (Belize)", "BOL (Bolivia)", "BRA (Brazil)", "BRB (Barbados)", "BRN (Brunei)", "BTN (Bhutan)", "BWA (Botswana)", "CAF (Central African Republic)", "CHL (Chile)", "CIV (Ivory Coast)", "CMR (Cameroon)", "COD (Democratic Republic of the Congo)", "COG (Republic of Congo)", "COL (Colombia)", "COM (Comoros)", "CPV (Cape Verde)", "CRI (Costa Rica)", "CUB (Cuba)", "CUW (Cura\u00e7ao)", "CYM (Cayman Islands)", "DJI (Djibouti)", "DMA (Dominica)", "DOM (Dominican Republic)", "DZA (Algeria)", "ECU (Ecuador)", "EGY (Egypt)", "ERI (Eritrea)", "ETH (Ethiopia)", "FLK (Falkland Islands)", "GAB (Gabon)", "GHA (Ghana)", "GIN (Guinea)", "GMB (Gambia)", "GNB (Guinea Bissau)", "GNQ (Equatorial Guinea)", "GRD (Grenada)", "GTM (Guatemala)", "GUY (Guyana)", "HND (Honduras)", "HTI (Haiti)", "IDN (Indonesia)", "IND (India)", "IOT (British Indian Ocean Territory)", "JAM (Jamaica)", "KEN (Kenya)", "KHM (Cambodia)", "KNA (Saint Kitts and Nevis)", "LAO (Laos)", "LBR (Liberia)", "LCA (Saint Lucia)", "LKA (Sri Lanka)", "LSO (Lesotho)", "MAF (Saint Martin)", "MDG (Madagascar)", "MDV (Maldives)", "MEX (Mexico)", "MOZ (Mozambique)", "MRT (Mauritania)", "MSR (Montserrat)", "MUS (Mauritius)", "MWI (Malawi)", "MYS (Malaysia)", "MYT (Mayotte)", "NAM (Namibia)", "NER (Niger)", "NGA (Nigeria)", "NIC (Nicaragua)", "NPL (Nepal)", "PAN (Panama)", "PER (Peru)", "PHL (Philippines)", "PRI (Puerto Rico)", "PRY (Paraguay)", "RWA (Rwanda)", "SDN (Sudan)", "SEN (Senegal)", "SGP (Singapore)", "SHN (Saint Helena)", "SLE (Sierra Leone)", "SLV (El Salvador)", "SOM (Somalia)", "STP (Sao Tome and Principe)", "SUR (Suriname)", "SWZ (Eswatini)", "SXM (Sint Maarten)", "SYC (Seychelles)", "TCA (Turks and Caicos Islands)", "TGO (Togo)", "THA (Thailand)", "TLS (East Timor)", "TTO (Trinidad and Tobago)", "TUN (Tunisia)", "TZA (United Republic of Tanzania)", "UGA (Uganda)", "URY (Uruguay)", "VCT (Saint Vincent and the Grenadines)", "VEN (Venezuela)", "VGB (British Virgin Islands)", "VIR (United States Virgin Islands)", "VNM (Vietnam)", "ZAF (South Africa)", "ZMB (Zambia)", "ZWE (Zimbabwe)"]
# @markdown **or** specify an area of interest in [WKT format](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) (assumes crs='EPSG:4326'); this [tool](https://arthur-e.github.io/Wicket/sandbox-gmaps3.html) might be useful.
your_own_wkt_polygon = ""  # @param {type:"string"}

# @markdown Second, specify which years to download
download_2016 = True  # @param { type: "boolean" }
download_2017 = True  # @param { type: "boolean" }
download_2018 = True  # @param { type: "boolean" }
download_2019 = True  # @param { type: "boolean" }
download_2020 = True  # @param { type: "boolean" }
download_2021 = True  # @param { type: "boolean" }
download_2022 = True  # @param { type: "boolean" }
download_2023 = True  # @param { type: "boolean" }

_GCS_BUCKET = "open-buildings-temporal-data"

_DATASET_VERSION = 'v1'

_GCS_MANIFESTS_FOLDER = "manifests"

_LOCAL_DOWNLOAD_URL_FILE_PATH = "/tmp/downloadable_urls.txt"

_MAX_NUM_THREADS = 8

_MANIFEST_S2_LEVEL = 2


def get_years_as_list() -> list[int]:
  years_to_download = []
  for year in range(2016, 2024):
    should_download = globals()[f"download_{year}"]
    if should_download:
      years_to_download.append(year)
  return years_to_download


def get_region_geometry(
    region_border_source: str, region: str, your_own_wkt_polygon: str
) -> shapely.geometry.base.BaseGeometry:
  """Returns the shapely geometry of the requested region."""

  if your_own_wkt_polygon:
    region_df = gpd.GeoDataFrame(
        geometry=gpd.GeoSeries.from_wkt([your_own_wkt_polygon]), crs="EPSG:4326"
    )
    if not isinstance(
        region_df.iloc[0].geometry, shapely.geometry.polygon.Polygon
    ) and not isinstance(
        region_df.iloc[0].geometry, shapely.geometry.multipolygon.MultiPolygon
    ):
      raise ValueError(
          "`your_own_wkt_polygon` must be a POLYGON or MULTIPOLYGON."
      )
    print(f"Preparing your_own_wkt_polygon.")
    return region_df.iloc[0].geometry

  if not region:
    raise ValueError("Please select a region or set your_own_wkt_polygon.")

  if region_border_source == "Natural Earth (Low Res 110m)":
    url = (
        "https://naciscdn.org/naturalearth/"
        "110m/cultural/ne_110m_admin_0_countries.zip"
    )
    !wget -N {url}
    display.clear_output()
    region_shapefile_path = os.path.basename(url)
  elif region_border_source == "Natural Earth (High Res 10m)":
    url = (
        "https://naciscdn.org/naturalearth/"
        "10m/cultural/ne_10m_admin_0_countries.zip"
    )
    !wget -N {url}
    display.clear_output()
    region_shapefile_path = os.path.basename(url)
  elif region_border_source == "World Bank (High Res 10m)":
    url = (
        "https://datacatalogfiles.worldbank.org/ddh-published/"
        "0038272/DR0046659/wb_countries_admin0_10m.zip"
    )
    !wget -N {url}
    !unzip -o {os.path.basename(url)}
    display.clear_output()
    region_shapefile_path = "WB_countries_Admin0_10m"

  region_iso_a3 = region.split(" ")[0]
  region_df = (
      gpd.read_file(region_shapefile_path)
      .query(f'ISO_A3 == "{region_iso_a3}"')
      .dissolve(by="ISO_A3")[["geometry"]]
  )
  print(f"Preparing {region} from {region_border_source}.")
  return region_df.iloc[0].geometry


def get_bounding_box_s2_covering_tokens(
    region_geometry: shapely.geometry.base.BaseGeometry,
) -> list[str]:
  """Returns the s2_tokens of the bounding box of the provided geometry."""
  region_bounds = region_geometry.bounds
  s2_lat_lng_rect = s2.S2LatLngRect_FromPointPair(
      s2.S2LatLng_FromDegrees(region_bounds[1], region_bounds[0]),
      s2.S2LatLng_FromDegrees(region_bounds[3], region_bounds[2]),
  )
  coverer = s2.S2RegionCoverer()
  # NOTE: Should be kept in-sync with manifest s2 cell level.
  coverer.set_fixed_level(_MANIFEST_S2_LEVEL)
  coverer.set_max_cells(1000000)
  return [cell.ToToken() for cell in coverer.GetCovering(s2_lat_lng_rect)]


def get_matching_manifest_blobs(s2_token: str) -> list[storage.Blob]:
  """Returns a list of manifest blobs for the given s2_token."""
  matching_manifest_blobs = []
  token_manifest_blobs = list(
      storage_client.list_blobs(
          _GCS_BUCKET,
          prefix=os.path.join(_DATASET_VERSION, _GCS_MANIFESTS_FOLDER, f'{s2_token}_'),
      )
  )
  for year in get_years_as_list():
    filtered_token_manifests = [
        blob for blob in token_manifest_blobs if f'_{str(year)}_' in blob.name
    ]
    matching_manifest_blobs.extend(filtered_token_manifests)
  return matching_manifest_blobs


def multithreaded_fn(progress_bar_desc: str, fn: Callable, items: Iterable[Any]):
  """Run `fn` on `items` using multithreading and display a progress bar."""
  total_num_items = len(items)
  fn_results = []
  with tqdm.notebook.tqdm(
      total=len(items), desc=progress_bar_desc
  ) as pbar:
    with ThreadPool(processes=_MAX_NUM_THREADS) as pool:
      for result in pool.map(fn, items):
        fn_results.extend(result)
        pbar.update(1)
  return fn_results



def multithreaded_get_matching_manifest_blobs(
    s2_tokens: list[str],
) -> list[storage.Blob]:
  """Returns a list of manifest blobs for the given s2_tokens."""
  return multithreaded_fn("Fetching matching manifests",
                          get_matching_manifest_blobs, s2_tokens)


def extract_tile_polygons(
    manifest_bytes: bytes,
) -> list[str]:
  """Extracts GeoTIFF urls from a manifest."""
  tile_polys = []
  manifest = json.loads(manifest_bytes)
  crs = None
  for tileset in manifest["tilesets"]:
    for source in tileset["sources"]:
      # All tiles in a manifest should have the same projection
      if crs is None:
        crs = tileset["crs"]
      affine_transform = source["affineTransform"]
      transform = Affine.translation(
          affine_transform["translateX"], affine_transform["translateY"]
      ) * Affine.scale(affine_transform["scaleX"], affine_transform["scaleY"])
      dimensions = source["dimensions"]
      width = dimensions["width"]
      height = dimensions["height"]

      corners = [(0, 0), (width, 0), (width, height), (0, height)]
      corners = [transform * corner for corner in corners]

      uri = source["uris"][0]
      object_path = manifest["uriPrefix"] + uri
      tile_polys.append((object_path, Polygon(corners)))

  return tile_polys, crs


def extract_geotiff_urls(
    manifest_blob: storage.Blob,
    region_geometry: shapely.geometry.base.BaseGeometry,
) -> list[str]:
  """Extracts GeoTIFF urls from a manifest intersecting `region_geometry`."""
  manifest_bytes = manifest_blob.download_as_bytes()
  tile_polys, crs = extract_tile_polygons(manifest_bytes)
  # EPSG:4326 is the standard WGS84 lat/lon coordinate system. We transform
  # region_geometry from EPSG:4326 to manifest's projection before doing
  # intersection check.
  transformer = pyproj.Transformer.from_crs("epsg:4326", crs, always_xy=True)
  region_geometry = transform(transformer.transform, region_geometry)
  geotiff_urls = []
  for (url, poly) in tile_polys:
    if poly.intersects(region_geometry):
      geotiff_urls.append(url)

  return geotiff_urls


def multithreaded_extract_geotiff_urls(
    manifest_blobs: list[storage.Blob],
    region_geometry: shapely.geometry.base.BaseGeometry,
) -> list[str]:
  """Extracts GeoTIFF urls from manifests."""
  return multithreaded_fn(
      "Extracting urls",
      lambda manifest_blob: extract_geotiff_urls(
          manifest_blob, region_geometry
      ),
      manifest_blobs,
  )


def write_to_file(filename: str, urls: list[str]) -> None:
  """Writes urls to file."""
  with open(filename, "w") as f:
    for url in urls:
      f.write(f"{url}\n")


# Clear output after pip install.
display.clear_output()
storage_client = storage.Client(credentials=credentials.AnonymousCredentials())
geometry = get_region_geometry(
    region_border_source, region, your_own_wkt_polygon
)
s2_tokens = get_bounding_box_s2_covering_tokens(geometry)

region_manifest_blobs = multithreaded_get_matching_manifest_blobs(s2_tokens)

geotiff_urls = multithreaded_extract_geotiff_urls(
    region_manifest_blobs, geometry
)

write_to_file(_LOCAL_DOWNLOAD_URL_FILE_PATH, geotiff_urls)

print(f"Finished writing urls to file. File contains {len(geotiff_urls)} urls")

In [ ]:
# @title Download text file with urls

from google.colab import files

files.download(_LOCAL_DOWNLOAD_URL_FILE_PATH)

file_name = os.path.basename(_LOCAL_DOWNLOAD_URL_FILE_PATH)
print(f"""
Download the GeoTIFFs using the gsutil command like so:

cat {file_name} | gsutil -m cp -I /my/target


Alternatively, GeoTIFFs can also be loaded in Earth Engine directly. Example: https://code.earthengine.google.com/ca39918365011141c154e9fa26b2c563

""")

In [ ]:
# Importa las bibliotecas necesarias
import ee
import os
from google.colab import drive

# Conecta Google Drive para almacenar los archivos descargados
drive.mount('/content/drive')

# Inicializa la API de Google Earth Engine
try:
    ee.Authenticate()
    ee.Initialize()
except Exception as e:
    print(f"Error al inicializar Google Earth Engine: {e}")
    raise


# Lista de URIs de las imágenes GeoTIFF
geotiff_uris = [
    'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_UlfQI_P7vco.tif',
    'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_G2nePRpiK6Q.tif',
    'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_jVMuoz3FmBA.tif',
    'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile__lHwd88WlAk.tif',
    'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_uA6JC01J-II.tif',
    'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_n6WL2zzxprU.tif',
    'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_e7OyCSqpV-c.tif',
    'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_MeDOXFSvFNs.tif',
    'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_uptv8iGJBi4.tif',
    'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_PiL6Joun_-M.tif',
    'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_j1y0l7fP-Sk.tif',
    'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_NYfDvlcPv0c.tif',
    'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_KhF1V8P1Z20.tif',
    'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_wcCkxNyBXr4.tif',
    'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_MJ1U3NpVqR8.tif'
]

# Carpeta de destino en Google Drive
output_folder = '/content/drive/My Drive/GeoTIFFs'
os.makedirs(output_folder, exist_ok=True)

# Función para descargar imágenes
for uri in geotiff_uris:
    try:
        # Cargar la imagen desde el URI
        image = ee.Image.loadGeoTIFF(uri)

        # Definir el nombre del archivo
        filename = uri.split('/')[-1]
        local_path = os.path.join(output_folder, filename)

        # Exportar la imagen como GeoTIFF
        print(f"Descargando: {filename}")
        ee.batch.Export.image.toDrive(
            image=image,
            description=filename,
            folder='GeoTIFFs',
            fileNamePrefix=filename.split('.')[0],
            scale=10,
            region=image.geometry().bounds()
        ).start()
    except Exception as e:
        print(f"Error descargando {uri}: {e}")

print(f"Las imágenes han sido exportadas a Google Drive en la carpeta {output_folder}.")


In [ ]:
# Importa las bibliotecas necesarias
import ee
import os
from google.colab import drive

# Conecta Google Drive para almacenar los archivos descargados
drive.mount('/content/drive')

# Inicializa la API de Google Earth Engine
# NOTE: You need to grant your google account the "roles/serviceusage.serviceUsageConsumer" role or a custom role with the "serviceusage.services.use" permission on the earthengine-legacy project in the Google Cloud Console: https://console.developers.google.com/iam-admin/iam/project?project=earthengine-legacy
ee.Authenticate()

In [ ]:
ee.Initialize(project='ee-mesendelgado')


In [ ]:

# Lista de URIs de las imágenes GeoTIFF
geotiff_uris = [
  'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_ncdQ401nuQM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_tnaJo4bzUSI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_3PP94IvU6LU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_OGCSC_GRupA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_xxz7CbfxxI0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_ovl1QE5MZxM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_-v3t1LPEWFE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_2UU7iRXs0pw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_Kaej5n7IJdI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_iM8BvsuXYgo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_bdveB5gKAR8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_Rr4IwM4hAR4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_dr5N5SqIUdc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_4WsINMFjQyA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_OuCHhfHiIiM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_m9WKjXykcrw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_za6lbJmtBNA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_zmVokDgk-Qc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_BvnC-jJY7wY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_n8T27F4DPvU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_t3tWUMfnM1s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile___2Ly15PD-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_N9fYMYCIDXM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_8u4vsdT3ezc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_zJgy7xJ6s14.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_XwIpcN0I42o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_KYwaBtt8bmc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_CgaoNS4tIPY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_BhG1_EGrxp0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_Dmbab34e-DM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_LoB42QzAKR4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_k_rYgCSlHoc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_Sm-4CiwChiU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_YdOJsuQ3NTE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_Wr5S4gpj6PI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_2sNXli1EiBM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_X4FrkiIFSFw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_oU9nU8lnzeU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_bu142fanRU4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_320NPFa_c6I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_ap6MmlOc2yU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_RnzVPV79IwM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_CjC2YokAYNM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_fvLCqLHC2gg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_fRnyS-6OCAI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_OWU6LzCOOSA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile__OvnbPSQ9ns.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_UXH8WmphVWQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2016_06_30/tile_CXR3m-7oxhw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_ldBb0Tr1ZwM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_rFvCO4Ah9no.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_5e0MejX9ZxU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_DBuHBh-X59s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_jgxruYmBwN8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_r8fBOaQdJ8E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_qGc0-6zuNLc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_Vdad6TSu3pw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_VRnX6WYY5aY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_FNTgkpAGzbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_2lmC2Q0_NGI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_31fe1gaUUJs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_oBoJG9S-IuU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_Qs4Ftn6U57k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_DzrfLr9c23M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_vQXO29VvamQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_YcYIvV8wU2o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_-lw3NKgt5CE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_eZ0UI9WkUa4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_GE7bCP4ZofQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_Dv6dcOWnI2s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_oWuYRxYPWts.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_9czOQj56LgE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_1jb0MV6ft44.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_GvjN6ss4IjM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_l2Ijk61lArY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_9_oKvfHROyI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_8prFLjh2fCA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_dvw7RgB7zV8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_oA1IXzGzBwU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_NmTWC35JRWA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2016_06_30/tile_EMjG3YF7g-U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2016_06_30/tile_VKlB1vsTo1k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2016_06_30/tile_iLYSjIvoV7c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2016_06_30/tile_g5NW4OyrF_w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2016_06_30/tile_UhcGGXD9HTA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2016_06_30/tile_IaKubeBxk-o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2016_06_30/tile_xlHKnJlfjjI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2016_06_30/tile_c5GJn9lqQlg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2016_06_30/tile_A1fOd0d6vfw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2016_06_30/tile_p8fmD97oUw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2016_06_30/tile_j-0-feR5x-Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2016_06_30/tile_D0Ykpy1Q3dE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2016_06_30/tile_fb49UA89wsE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2016_06_30/tile_VxTjtzLnz5s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2016_06_30/tile_MjZ7ijC2MMQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2016_06_30/tile_tI_DZyIguZw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2016_06_30/tile_2CBafV7FlMU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_F7op2ob98HQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_YuT9ce3zbXY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_dy3csCGSuNw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_BgGCZSUVKmE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_S9Z4shzmIsE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_vDGSmkdV7os.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_LpZkF1XEY3o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_vI4LhyMPWeg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_YPV1DLqSPIM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_notmacDKViI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_SyLdYKgXsIo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_x7Kq-k7XyAE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_3Q2vR4iJkm8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_sfCXwt6jEz8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_mSVN8yZI-Uk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_h_aPHCxQxSA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_iAKMUFyZAgE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_fSa4zxMdvmc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_szHgiDfUXTY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_66zEbCqh7LA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_HppyIg2uSME.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_Ry0ghFfa2jo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_Dvi1vUuf3Es.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_uahk6r8D0Wo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_2XHdvyceIE0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_oDQaqWtZuxs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_sdYsbVcgUh8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_yqFpuXzmrnw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_n8iP_OQUSLw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_gXWjVVfGCfk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_EV4_heFBtbs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_iNLBAvgkbTg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_P_yj4kitRUw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_eVz79CwsG7U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_oTzNnOPQcss.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_FJlshPGyBoo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_bvz58Sj6DgU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_xGCy64-7DpE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_d-Gf9Hjsfq8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_PDeI1LBjxz8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_gyMYOk8CKtY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_9YDjRVGgNDU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_W-lHKI0gAro.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_iWvpRfo_44w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_lhMLAAYgmBE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_5KlVTvp3Ud8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_26937ENrox4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_4NozDtFdows.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2016_06_30/tile_pgF_G4DWa8E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2016_06_30/tile_6l8nQqpP3Fs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2016_06_30/tile_Fvzd5OeOzH8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2016_06_30/tile_oJJJQx20xus.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2016_06_30/tile_ndb8nuuMFQE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2016_06_30/tile_xyE-tv1vbpg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2016_06_30/tile_fJmNd-vIC8I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2016_06_30/tile_uyjeDqZaf00.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2016_06_30/tile_ohY41G3cspQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2016_06_30/tile_8bxtBmYoeig.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2016_06_30/tile_r3cl8mRaJw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2016_06_30/tile_aOIVRthWYlM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2016_06_30/tile_ca5H2wWNgOk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2016_06_30/tile_7IbjBqHuIng.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2016_06_30/tile_sYxHTLvk0ZQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_0A2gJ29ZmZM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_ZhnhFasyWJQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_Af0yq0hni6I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_Fy_29KcjBV4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_DyawyUvpKNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_gUpSW8_n9tk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_WMfRllKRxBs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_lfAYMhyogzA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_b3D3Gy-xk54.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_414GXSY0u0k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_AfMu6nphNKM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_xxXoMLneFKQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_lO3dSzyBHVc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_da5aF9PdeiY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_7Zvh8BTRDDM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_Ei6ZwffbYpg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_BGo4763ZAe4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_-mIbXBC7-fM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_C55GT80DLy4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_zZUrRAwJHcs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_rfD3ZaPrkvA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_LLfSbGafW2s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_6GEy0NhrfLs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2016_06_30/tile_oC7JciLmHEI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_edtHlMNdHlk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_5bJrF7sLxoY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_X2U3Wb9Yt2w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_ZMuqD-gMYds.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_M5R2IAMwhZw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_3At_utvBFVU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_yD-5IqzlIt4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_zlZZ6C20Xiw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_hrPqtg2pzLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_z5EXKgocvxo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_4EjvsUke-gg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_ZiMtkI2Qi3c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_hk2xZot2PXE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_0WuKT_oH-c4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_zUe6xvSLPZ8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_ra5oBq3ZhqM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_tAS97RqKbsY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_Qmj1dsXHujU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_ThrufapKc8c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_-S4mAuWhX0s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_a2Ceu-awLLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_Qf2T9y2X8NQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_bQpozx7M-lU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_cek61MplDpc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_fTXx7GhMV_M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_cFkjr0PDIQg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_fnUcJei0LlA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_F43y26WmrH4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_z4UdTx58XLg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_SYDTGycWzaI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_hN6DkXzYW2M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_P0LL9oS_HA8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_YBzNNWaiQTw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_gOzae0pJwvs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2016_06_30/tile_4o-b8zJb58o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9ec_2016_06_30/tile_o55i51eydaQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_-PV0lk8Wcy0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_BbdgT1OZ4xA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_OebMUMSKp58.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_xAHpfDS_tcY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_Zz62K_GjAEg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_9P8rv_pKqQo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_CiL8thFOPLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_SWbvqMiMruQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_qOmIXpDy4tI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_GalsFR1ogcE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_Xd4BfprZwWo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_KQNKVCrMrew.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_ziyY0HB_UIk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_jOY4cOkLlOI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_Zo-kK8qGZ_A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_At-ZNnlNAdc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2016_06_30/tile_bo1ZdxoCCwQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2016_06_30/tile_rMqJeLTuHRE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2016_06_30/tile_nfD2LYI6nYo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2016_06_30/tile_Hjd3hvqGQ1s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2016_06_30/tile_Z37cKCmA-cM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2016_06_30/tile_cFdPOG9KpSE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2016_06_30/tile_GdNxjPqD76I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2016_06_30/tile_o4dsAI5yWBg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2016_06_30/tile_F6QKTYMZ3wI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2016_06_30/tile_5tWtjSZGz_k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2016_06_30/tile_gaHpMg_bTWU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_9z5qs7azJEg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_UeFS52yPtVE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_-UHrdUFyL5Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_jU67INtOoPE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_ibeuqkRRCDI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile__5iKyN8m6Og.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_dvywPun0dsA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_VWJc22h_hBQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_8_DRwoUsU9Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_NIVD-Z4zMU8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_w4XApkuShgs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_1yMS-7nDS5s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_lhR5sXnbzRk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_xhU82Q9ccg8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_GlSCj7IFwnY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_P_B1sisDlRY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_eXoQAjVcOYQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_vW2MJxoFgMk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_Sq3-rJUxsNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_xQ-44Zay7aA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_xDIcGmh6HVg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_CWZrLuhZaro.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_I67YZsLB-kw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_s9WQ6imuVJI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_Wdo7UBfds04.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_kYInGFjnjWA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_NGJz-evoHWQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_I_aWqMwULzI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2016_06_30/tile_IN36N6xwaq0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa34_2016_06_30/tile_OAmzV52YF2w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa34_2016_06_30/tile_20SedX5wtg8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_bIYrX_YAxwk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_Hu5c9sSMr3A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_vrjZGrwBCA0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_unQMAvozNRk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_0xI9fVduyZE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_wzA4WQgAvrE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_Y8ImHQMAcYA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_58pnWEGSKt8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_yY2x2fammoM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_8nVOcGNu2IA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_RlsKT6Jf6wM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_pq8UH-HEOQg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_VqOmW5kz1W0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_PKm8xyhmIZc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_b_WBA1b7eZA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_FeeFcJAZbYc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_fWsQe-ZeEe8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_UPxsMf5HJWI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_SVIOaY507UM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_QM9EXlxbeBA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_a9oLR3z8Vm0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_R84NkdXV2Z0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_nNA6Pwk-M3U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_jxm7UNj7-Rc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_rc_yujmCW-U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_2OIs0gqeq1k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_Gex4QzSIKy0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_Hc-UpwT69ws.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_gVc0TpxZv7E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_MISuRqfSgeo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_fvrUa3e43x4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_I5fa0cA523A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_JQxlFUS4NfQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_vdnHmna7Ifo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_NmJNE6dWWos.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_xHWoK3gkHe8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_u8FFj7cZHIQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_QWcgi3AtUuU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_4kQAI2-RaC4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_ROOd-NnESME.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_R4y_itQ4hd4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_VFjB7PY-h3w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_MFmAF07BBrc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_F14sWKylN6c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_qUphd3XFpW0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2016_06_30/tile_QtvRPAbpGLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2016_06_30/tile_RxMvIrdiSa4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2016_06_30/tile_eJzTsOJZsUg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2016_06_30/tile_cOL6HYvY8VA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_878QAjksX9k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_5AGXayBwZCM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_95lAfaJ_1o4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_RnIje33rRLY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_iFHrjLE08Vg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_xYnut80Y-zE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_HdPOmocD19w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_UlfQI_P7vco.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_G2nePRpiK6Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_jVMuoz3FmBA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile__lHwd88WlAk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_uA6JC01J-II.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_n6WL2zzxprU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_e7OyCSqpV-c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_MeDOXFSvFNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_uptv8iGJBi4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_PiL6Joun_-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_j1y0l7fP-Sk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_NYfDvlcPv0c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_KhF1V8P1Z20.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_wcCkxNyBXr4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2016_06_30/tile_MJ1U3NpVqR8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_VPcv0zeEUUY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_Qgqg0VZsvoo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_l6o10K2govA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_2fJYus8UkwQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_-ElEld5rt5E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_iZkKS0q50w4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_gS5D6klKAkE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_EtmfDSzhAw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_hqTAUj8QqPw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_88jQlqXOVf8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_mgmMDaDtiUQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_nDRZTLCQqvI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_6ViRnrkFi_Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_4bKe6c_tbQo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_RjKOUXqeAsU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_cfa0maP-1FE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_mcPmzf75ToE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_gUx9RYKOj8w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_p_4EDz0m5iU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile__iS3cAV0l1Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_F66qhzaKdUg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_aIt-UR2UqDE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_0chzaXGxohY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_xDRoay6WAbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_T-EVYoBtsvk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_1_M0gDok2-0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_GcKlG8oRV6o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_6RRVAfi_A6Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_3SoNvBqku9E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_iDsfCsFq_II.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_MpAXOG-IkOs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_IdsknuIsqQE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_YcFMiBDiD60.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_6pQK9PWLHRw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_BEpPJrteTbg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_Zw2iwVxp9ok.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_jDOBX14ESqY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_YBUOyhJE9C0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_OC2WfdcNv7I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_atQiFq44XNY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_dFh6u3eGS14.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_EbYnhHZcsoM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_HEYB6rB4afs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_ufajWPtQrts.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_qrp1SrVUIic.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_sCoH8x5AbKI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_mCqEfn-4q4A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_PqI_YrPNSWE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2016_06_30/tile_m6dYq8wGj5U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_nPkNNFAZKLM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_9v8P-w1N6_g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_LCXbkT5Dbv4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_rO4GFLbHNqk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_SjJwsggAeY4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_HwzR7aY6WIo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_zikBG5UPINA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_-0lEnLgfrGU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_VyEdJOyGwNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_JyXtw1WXzu4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile__qWrETqMx48.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_hv51HgzPQjc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_DWzHgPsDu_c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_CRQSl5Iz7Rk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_TcomU2q_loQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_bSSp3yNUssU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_EtzjtiiE4ZY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_siUk0QP_z8g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_otJdZNEr8X8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_m_g0A3UNuGY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_rrzf2r3R5Js.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_os-FTpdD5sc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_lAXVJ1CZ8PE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_f6mhLCvACJk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_hGDdIkHOdLU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_MUTAkRKvdHI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_EdNson2UbQc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_GTMv-a6-QQI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_Me2UovXVGlw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_kVYBvRC38zs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_FUN5e6yclBM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_whdVajibyDI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_igkH8p44L3U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_obdfPw2Hu3s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_6ITL_yjNmww.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_TvXw-dGPgqA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_kjMdbwQjorw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_bfkzc-Hocc0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_xU55vI4txgU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_iELDzowUH7M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_XhrqU2iikHQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_BiuWipHcYKQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2016_06_30/tile_v0HPhTzrt20.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_bvvyF6CZ3v0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_YJmq-27yWK4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_N77B5UsvXQI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_0qjzxzBybu8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_Cfv04geebOc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_wea4hDdk5sM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_oaUjV8M2qVY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_yzO245pSh50.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_YxKi3PRdpE8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_98ZSThIgVNw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_UbaNaWeQQhY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_uys4W-tzfws.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_-6WIgLec_Aw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_uZXb1B-6-Pc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_zaqnXBBelc4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_r9EbIgjkq3Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_pdkAbSnbGXM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2016_06_30/tile_V4S8RzrAoRM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2016_06_30/tile_I1OMvjAHoc8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2016_06_30/tile_MVyILXNgnas.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2016_06_30/tile_dPY0-2oQmtU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2016_06_30/tile_LBRA9z5hNF4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2016_06_30/tile_XacG0jFDyH8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2016_06_30/tile_5WXILtuOQbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2016_06_30/tile_uedmM83ZqRs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2016_06_30/tile_H42ZeH4gFtI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2016_06_30/tile_4PJBpAUrAqw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2016_06_30/tile_y4YR_QsaTcQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2016_06_30/tile_72OrTmz8LwY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2016_06_30/tile_45CwXin5MhE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2016_06_30/tile_rxgV4aC8GWg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2016_06_30/tile_hn2htg6Y1KQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2016_06_30/tile_R6kRjlEAS2Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2016_06_30/tile_PZ7IGn7F_pE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2016_06_30/tile_8AHkyfOa04w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2016_06_30/tile_25mk9l1sSUU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2016_06_30/tile_ezNXcekn93E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2016_06_30/tile_xV-T0Fy3KFU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2016_06_30/tile_zcYd-KLAVFM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa7c_2016_06_30/tile_8Eo7jYPCsEE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2016_06_30/tile_6RNE_2tn9Z0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2016_06_30/tile_cI_89vaQokA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2016_06_30/tile_5sFxLBA_Iqk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2016_06_30/tile_SrA3hNKGZ64.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2016_06_30/tile_Dd4GpvS8Ge0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2016_06_30/tile_YNgafwyhidk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2016_06_30/tile_t8LbcNrkWrE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2016_06_30/tile_SCFPnVfUql8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2016_06_30/tile_xVbV88yWUhE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_PGmIEWWtvYg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_lrb8RyXXxUs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_crBc3qKvc_c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_pC6GZFISPXI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_jxnI9CDibDk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_HxMnqi_eHyc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_HwWmPLz2-Vc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_U1sP7RWC1gA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_oHUY41RqYrU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_8qViUzHeLCw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_MuJoJ-9MANU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_-Cuet1TxnNo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_FrFxz3X-7Mo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_AWio5XBXSAc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_9dX3Ngf4Z-4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_gzcQRuzEqLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_0J1CoSS6iPg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_scIfdKTURXA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_ap0j3H6RE1g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile__oLuYSkg12A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_zOGOLNWhXco.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_x73vJhmPBaI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_4Wt4RqUDYv8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_3DxVMYeRslw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_bJ2k97ym_Kw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_yIGOSSIzoMg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_Ig2W_gviuTI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_1YAFw6W52-E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_TF4q8zPmRRc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_mJYLHk7hDEc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_2BM8UBy62go.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_THLy2cMWTJE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_F68y1CtJLsg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_YevJTKbTzTw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_X7Vdb4oIr2Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_xaFmlC3lb-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_DoySuYmtk-w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_KYDEwg89lH0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_vy7ZbKCC8Lk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_W2vgIrSqXIA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_0C3XV_w3Hcc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_U8Xo8TRr30Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_H-lLEvnmoLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_7-krjk7IdVY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_Fsy2GZyiqT8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_Wy2x2VSavKU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_ZxrWWCThOi4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2016_06_30/tile_GSz_zarI5kI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2017_06_30/tile_6l8nQqpP3Fs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2017_06_30/tile_Fvzd5OeOzH8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2017_06_30/tile_oJJJQx20xus.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2017_06_30/tile_ndb8nuuMFQE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2017_06_30/tile_xyE-tv1vbpg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2017_06_30/tile_fJmNd-vIC8I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2017_06_30/tile_uyjeDqZaf00.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2017_06_30/tile_ohY41G3cspQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2017_06_30/tile_8bxtBmYoeig.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2017_06_30/tile_r3cl8mRaJw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2017_06_30/tile_aOIVRthWYlM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2017_06_30/tile_ca5H2wWNgOk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2017_06_30/tile_7IbjBqHuIng.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2017_06_30/tile_sYxHTLvk0ZQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_edtHlMNdHlk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_5bJrF7sLxoY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_X2U3Wb9Yt2w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_ZMuqD-gMYds.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_M5R2IAMwhZw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_3At_utvBFVU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_yD-5IqzlIt4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_zlZZ6C20Xiw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_hrPqtg2pzLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_z5EXKgocvxo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_4EjvsUke-gg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_ZiMtkI2Qi3c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_hk2xZot2PXE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_0WuKT_oH-c4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_zUe6xvSLPZ8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_ra5oBq3ZhqM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_tAS97RqKbsY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_Qmj1dsXHujU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_ThrufapKc8c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_-S4mAuWhX0s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_a2Ceu-awLLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_Qf2T9y2X8NQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_bQpozx7M-lU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_cek61MplDpc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_fTXx7GhMV_M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_cFkjr0PDIQg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_fnUcJei0LlA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_F43y26WmrH4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_z4UdTx58XLg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_SYDTGycWzaI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_hN6DkXzYW2M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_P0LL9oS_HA8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_YBzNNWaiQTw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_gOzae0pJwvs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2017_06_30/tile_4o-b8zJb58o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_F7op2ob98HQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_YuT9ce3zbXY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_dy3csCGSuNw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_BgGCZSUVKmE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_S9Z4shzmIsE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_vDGSmkdV7os.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_LpZkF1XEY3o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_vI4LhyMPWeg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_YPV1DLqSPIM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_notmacDKViI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_SyLdYKgXsIo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_x7Kq-k7XyAE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_3Q2vR4iJkm8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_sfCXwt6jEz8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_mSVN8yZI-Uk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_h_aPHCxQxSA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_iAKMUFyZAgE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_fSa4zxMdvmc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_szHgiDfUXTY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_66zEbCqh7LA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_HppyIg2uSME.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_Ry0ghFfa2jo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_Dvi1vUuf3Es.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_uahk6r8D0Wo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_2XHdvyceIE0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_oDQaqWtZuxs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_sdYsbVcgUh8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_yqFpuXzmrnw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_n8iP_OQUSLw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_gXWjVVfGCfk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_EV4_heFBtbs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_iNLBAvgkbTg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_P_yj4kitRUw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_eVz79CwsG7U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_oTzNnOPQcss.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_FJlshPGyBoo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_bvz58Sj6DgU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_xGCy64-7DpE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_d-Gf9Hjsfq8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_PDeI1LBjxz8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_gyMYOk8CKtY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_9YDjRVGgNDU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_W-lHKI0gAro.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_iWvpRfo_44w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_lhMLAAYgmBE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_5KlVTvp3Ud8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_26937ENrox4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_4NozDtFdows.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2017_06_30/tile_pgF_G4DWa8E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_ncdQ401nuQM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_tnaJo4bzUSI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_3PP94IvU6LU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_OGCSC_GRupA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_xxz7CbfxxI0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_ovl1QE5MZxM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_-v3t1LPEWFE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_2UU7iRXs0pw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_Kaej5n7IJdI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_iM8BvsuXYgo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_bdveB5gKAR8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_Rr4IwM4hAR4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_dr5N5SqIUdc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_4WsINMFjQyA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_OuCHhfHiIiM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_m9WKjXykcrw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_za6lbJmtBNA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_zmVokDgk-Qc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_BvnC-jJY7wY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_n8T27F4DPvU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_t3tWUMfnM1s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile___2Ly15PD-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_N9fYMYCIDXM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_8u4vsdT3ezc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_zJgy7xJ6s14.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_XwIpcN0I42o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_KYwaBtt8bmc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_CgaoNS4tIPY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_BhG1_EGrxp0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_Dmbab34e-DM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_LoB42QzAKR4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_k_rYgCSlHoc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_Sm-4CiwChiU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_YdOJsuQ3NTE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_Wr5S4gpj6PI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_2sNXli1EiBM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_X4FrkiIFSFw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_oU9nU8lnzeU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_bu142fanRU4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_320NPFa_c6I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_ap6MmlOc2yU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_RnzVPV79IwM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_CjC2YokAYNM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_fvLCqLHC2gg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_fRnyS-6OCAI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_OWU6LzCOOSA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile__OvnbPSQ9ns.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_UXH8WmphVWQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2017_06_30/tile_CXR3m-7oxhw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_-PV0lk8Wcy0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_BbdgT1OZ4xA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_OebMUMSKp58.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_xAHpfDS_tcY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_Zz62K_GjAEg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_9P8rv_pKqQo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_CiL8thFOPLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_SWbvqMiMruQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_qOmIXpDy4tI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_GalsFR1ogcE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_Xd4BfprZwWo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_KQNKVCrMrew.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_ziyY0HB_UIk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_jOY4cOkLlOI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_Zo-kK8qGZ_A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_At-ZNnlNAdc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2017_06_30/tile_bo1ZdxoCCwQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9ec_2017_06_30/tile_o55i51eydaQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_0A2gJ29ZmZM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_ZhnhFasyWJQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_Af0yq0hni6I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_Fy_29KcjBV4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_DyawyUvpKNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_gUpSW8_n9tk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_WMfRllKRxBs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_lfAYMhyogzA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_b3D3Gy-xk54.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_414GXSY0u0k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_AfMu6nphNKM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_xxXoMLneFKQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_lO3dSzyBHVc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_da5aF9PdeiY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_7Zvh8BTRDDM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_Ei6ZwffbYpg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_BGo4763ZAe4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_-mIbXBC7-fM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_C55GT80DLy4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_zZUrRAwJHcs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_rfD3ZaPrkvA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_LLfSbGafW2s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_6GEy0NhrfLs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2017_06_30/tile_oC7JciLmHEI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2017_06_30/tile_VKlB1vsTo1k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2017_06_30/tile_iLYSjIvoV7c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2017_06_30/tile_g5NW4OyrF_w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2017_06_30/tile_UhcGGXD9HTA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2017_06_30/tile_IaKubeBxk-o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2017_06_30/tile_xlHKnJlfjjI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2017_06_30/tile_c5GJn9lqQlg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2017_06_30/tile_A1fOd0d6vfw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2017_06_30/tile_p8fmD97oUw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2017_06_30/tile_j-0-feR5x-Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2017_06_30/tile_D0Ykpy1Q3dE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2017_06_30/tile_fb49UA89wsE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2017_06_30/tile_VxTjtzLnz5s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2017_06_30/tile_MjZ7ijC2MMQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2017_06_30/tile_tI_DZyIguZw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2017_06_30/tile_2CBafV7FlMU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_ldBb0Tr1ZwM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_rFvCO4Ah9no.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_5e0MejX9ZxU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_DBuHBh-X59s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_jgxruYmBwN8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_r8fBOaQdJ8E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_qGc0-6zuNLc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_Vdad6TSu3pw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_VRnX6WYY5aY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_FNTgkpAGzbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_2lmC2Q0_NGI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_31fe1gaUUJs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_oBoJG9S-IuU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_Qs4Ftn6U57k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_DzrfLr9c23M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_vQXO29VvamQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_YcYIvV8wU2o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_-lw3NKgt5CE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_eZ0UI9WkUa4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_GE7bCP4ZofQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_Dv6dcOWnI2s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_oWuYRxYPWts.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_9czOQj56LgE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_1jb0MV6ft44.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_GvjN6ss4IjM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_l2Ijk61lArY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_9_oKvfHROyI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_8prFLjh2fCA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_dvw7RgB7zV8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_oA1IXzGzBwU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_NmTWC35JRWA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2017_06_30/tile_EMjG3YF7g-U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa7c_2017_06_30/tile_8Eo7jYPCsEE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_PGmIEWWtvYg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_lrb8RyXXxUs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_crBc3qKvc_c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_pC6GZFISPXI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_jxnI9CDibDk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_HxMnqi_eHyc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_HwWmPLz2-Vc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_U1sP7RWC1gA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_oHUY41RqYrU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_8qViUzHeLCw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_MuJoJ-9MANU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_-Cuet1TxnNo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_FrFxz3X-7Mo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_AWio5XBXSAc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_9dX3Ngf4Z-4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_gzcQRuzEqLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_0J1CoSS6iPg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_scIfdKTURXA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_ap0j3H6RE1g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile__oLuYSkg12A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_zOGOLNWhXco.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_x73vJhmPBaI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_4Wt4RqUDYv8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_3DxVMYeRslw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_bJ2k97ym_Kw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_yIGOSSIzoMg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_Ig2W_gviuTI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_1YAFw6W52-E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_TF4q8zPmRRc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_mJYLHk7hDEc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_2BM8UBy62go.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_THLy2cMWTJE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_F68y1CtJLsg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_YevJTKbTzTw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_X7Vdb4oIr2Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_xaFmlC3lb-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_DoySuYmtk-w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_KYDEwg89lH0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_vy7ZbKCC8Lk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_W2vgIrSqXIA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_0C3XV_w3Hcc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_U8Xo8TRr30Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_H-lLEvnmoLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_7-krjk7IdVY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_Fsy2GZyiqT8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_Wy2x2VSavKU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_ZxrWWCThOi4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2017_06_30/tile_GSz_zarI5kI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2017_06_30/tile_6RNE_2tn9Z0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2017_06_30/tile_cI_89vaQokA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2017_06_30/tile_5sFxLBA_Iqk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2017_06_30/tile_SrA3hNKGZ64.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2017_06_30/tile_Dd4GpvS8Ge0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2017_06_30/tile_YNgafwyhidk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2017_06_30/tile_t8LbcNrkWrE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2017_06_30/tile_SCFPnVfUql8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2017_06_30/tile_xVbV88yWUhE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_bvvyF6CZ3v0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_YJmq-27yWK4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_N77B5UsvXQI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_0qjzxzBybu8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_Cfv04geebOc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_wea4hDdk5sM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_oaUjV8M2qVY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_yzO245pSh50.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_YxKi3PRdpE8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_98ZSThIgVNw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_UbaNaWeQQhY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_uys4W-tzfws.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_-6WIgLec_Aw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_uZXb1B-6-Pc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_zaqnXBBelc4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_r9EbIgjkq3Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_pdkAbSnbGXM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2017_06_30/tile_V4S8RzrAoRM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_878QAjksX9k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_5AGXayBwZCM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_95lAfaJ_1o4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_RnIje33rRLY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_iFHrjLE08Vg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_xYnut80Y-zE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_HdPOmocD19w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_UlfQI_P7vco.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_G2nePRpiK6Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_jVMuoz3FmBA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile__lHwd88WlAk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_uA6JC01J-II.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_n6WL2zzxprU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_e7OyCSqpV-c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_MeDOXFSvFNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_uptv8iGJBi4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_PiL6Joun_-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_j1y0l7fP-Sk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_NYfDvlcPv0c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_KhF1V8P1Z20.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_wcCkxNyBXr4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2017_06_30/tile_MJ1U3NpVqR8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_9z5qs7azJEg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_UeFS52yPtVE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_-UHrdUFyL5Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_jU67INtOoPE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_ibeuqkRRCDI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile__5iKyN8m6Og.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_dvywPun0dsA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_VWJc22h_hBQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_8_DRwoUsU9Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_NIVD-Z4zMU8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_w4XApkuShgs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_1yMS-7nDS5s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_lhR5sXnbzRk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_xhU82Q9ccg8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_GlSCj7IFwnY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_P_B1sisDlRY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_eXoQAjVcOYQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_vW2MJxoFgMk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_Sq3-rJUxsNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_xQ-44Zay7aA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_xDIcGmh6HVg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_CWZrLuhZaro.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_I67YZsLB-kw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_s9WQ6imuVJI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_Wdo7UBfds04.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_kYInGFjnjWA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_NGJz-evoHWQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_I_aWqMwULzI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2017_06_30/tile_IN36N6xwaq0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2017_06_30/tile_8AHkyfOa04w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2017_06_30/tile_25mk9l1sSUU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2017_06_30/tile_ezNXcekn93E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2017_06_30/tile_xV-T0Fy3KFU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2017_06_30/tile_zcYd-KLAVFM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_nPkNNFAZKLM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_9v8P-w1N6_g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_LCXbkT5Dbv4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_rO4GFLbHNqk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_SjJwsggAeY4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_HwzR7aY6WIo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_zikBG5UPINA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_-0lEnLgfrGU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_VyEdJOyGwNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_JyXtw1WXzu4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile__qWrETqMx48.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_hv51HgzPQjc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_DWzHgPsDu_c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_CRQSl5Iz7Rk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_TcomU2q_loQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_bSSp3yNUssU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_EtzjtiiE4ZY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_siUk0QP_z8g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_otJdZNEr8X8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_m_g0A3UNuGY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_rrzf2r3R5Js.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_os-FTpdD5sc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_lAXVJ1CZ8PE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_f6mhLCvACJk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_hGDdIkHOdLU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_MUTAkRKvdHI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_EdNson2UbQc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_GTMv-a6-QQI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_Me2UovXVGlw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_kVYBvRC38zs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_FUN5e6yclBM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_whdVajibyDI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_igkH8p44L3U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_obdfPw2Hu3s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_6ITL_yjNmww.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_TvXw-dGPgqA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_kjMdbwQjorw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_bfkzc-Hocc0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_xU55vI4txgU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_iELDzowUH7M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_XhrqU2iikHQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_BiuWipHcYKQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2017_06_30/tile_v0HPhTzrt20.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2017_06_30/tile_RxMvIrdiSa4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2017_06_30/tile_eJzTsOJZsUg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2017_06_30/tile_cOL6HYvY8VA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_bIYrX_YAxwk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_Hu5c9sSMr3A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_vrjZGrwBCA0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_unQMAvozNRk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_0xI9fVduyZE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_wzA4WQgAvrE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_Y8ImHQMAcYA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_58pnWEGSKt8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_yY2x2fammoM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_8nVOcGNu2IA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_RlsKT6Jf6wM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_pq8UH-HEOQg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_VqOmW5kz1W0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_PKm8xyhmIZc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_b_WBA1b7eZA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_FeeFcJAZbYc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_fWsQe-ZeEe8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_UPxsMf5HJWI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_SVIOaY507UM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_QM9EXlxbeBA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_a9oLR3z8Vm0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_R84NkdXV2Z0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_nNA6Pwk-M3U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_jxm7UNj7-Rc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_rc_yujmCW-U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_2OIs0gqeq1k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_Gex4QzSIKy0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_Hc-UpwT69ws.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_gVc0TpxZv7E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_MISuRqfSgeo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_fvrUa3e43x4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_I5fa0cA523A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_JQxlFUS4NfQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_vdnHmna7Ifo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_NmJNE6dWWos.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_xHWoK3gkHe8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_u8FFj7cZHIQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_QWcgi3AtUuU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_4kQAI2-RaC4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_ROOd-NnESME.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_R4y_itQ4hd4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_VFjB7PY-h3w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_MFmAF07BBrc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_F14sWKylN6c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_qUphd3XFpW0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2017_06_30/tile_QtvRPAbpGLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa34_2017_06_30/tile_OAmzV52YF2w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa34_2017_06_30/tile_20SedX5wtg8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2017_06_30/tile_rMqJeLTuHRE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2017_06_30/tile_nfD2LYI6nYo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2017_06_30/tile_Hjd3hvqGQ1s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2017_06_30/tile_Z37cKCmA-cM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2017_06_30/tile_cFdPOG9KpSE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2017_06_30/tile_GdNxjPqD76I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2017_06_30/tile_o4dsAI5yWBg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2017_06_30/tile_F6QKTYMZ3wI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2017_06_30/tile_5tWtjSZGz_k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2017_06_30/tile_gaHpMg_bTWU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_VPcv0zeEUUY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_Qgqg0VZsvoo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_l6o10K2govA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_2fJYus8UkwQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_-ElEld5rt5E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_iZkKS0q50w4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_gS5D6klKAkE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_EtmfDSzhAw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_hqTAUj8QqPw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_88jQlqXOVf8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_mgmMDaDtiUQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_nDRZTLCQqvI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_6ViRnrkFi_Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_4bKe6c_tbQo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_RjKOUXqeAsU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_cfa0maP-1FE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_mcPmzf75ToE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_gUx9RYKOj8w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_p_4EDz0m5iU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile__iS3cAV0l1Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_F66qhzaKdUg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_aIt-UR2UqDE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_0chzaXGxohY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_xDRoay6WAbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_T-EVYoBtsvk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_1_M0gDok2-0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_GcKlG8oRV6o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_6RRVAfi_A6Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_3SoNvBqku9E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_iDsfCsFq_II.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_MpAXOG-IkOs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_IdsknuIsqQE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_YcFMiBDiD60.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_6pQK9PWLHRw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_BEpPJrteTbg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_Zw2iwVxp9ok.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_jDOBX14ESqY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_YBUOyhJE9C0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_OC2WfdcNv7I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_atQiFq44XNY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_dFh6u3eGS14.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_EbYnhHZcsoM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_HEYB6rB4afs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_ufajWPtQrts.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_qrp1SrVUIic.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_sCoH8x5AbKI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_mCqEfn-4q4A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_PqI_YrPNSWE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2017_06_30/tile_m6dYq8wGj5U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2017_06_30/tile_I1OMvjAHoc8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2017_06_30/tile_MVyILXNgnas.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2017_06_30/tile_dPY0-2oQmtU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2017_06_30/tile_LBRA9z5hNF4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2017_06_30/tile_XacG0jFDyH8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2017_06_30/tile_5WXILtuOQbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2017_06_30/tile_uedmM83ZqRs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2017_06_30/tile_H42ZeH4gFtI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2017_06_30/tile_4PJBpAUrAqw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2017_06_30/tile_y4YR_QsaTcQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2017_06_30/tile_72OrTmz8LwY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2017_06_30/tile_45CwXin5MhE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2017_06_30/tile_rxgV4aC8GWg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2017_06_30/tile_hn2htg6Y1KQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2017_06_30/tile_R6kRjlEAS2Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2017_06_30/tile_PZ7IGn7F_pE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9ec_2018_06_30/tile_o55i51eydaQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_ncdQ401nuQM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_tnaJo4bzUSI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_3PP94IvU6LU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_OGCSC_GRupA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_xxz7CbfxxI0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_ovl1QE5MZxM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_-v3t1LPEWFE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_2UU7iRXs0pw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_Kaej5n7IJdI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_iM8BvsuXYgo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_bdveB5gKAR8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_Rr4IwM4hAR4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_dr5N5SqIUdc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_4WsINMFjQyA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_OuCHhfHiIiM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_m9WKjXykcrw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_za6lbJmtBNA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_zmVokDgk-Qc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_BvnC-jJY7wY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_n8T27F4DPvU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_t3tWUMfnM1s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile___2Ly15PD-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_N9fYMYCIDXM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_8u4vsdT3ezc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_zJgy7xJ6s14.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_XwIpcN0I42o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_KYwaBtt8bmc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_CgaoNS4tIPY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_BhG1_EGrxp0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_Dmbab34e-DM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_LoB42QzAKR4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_k_rYgCSlHoc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_Sm-4CiwChiU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_YdOJsuQ3NTE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_Wr5S4gpj6PI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_2sNXli1EiBM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_X4FrkiIFSFw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_oU9nU8lnzeU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_bu142fanRU4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_320NPFa_c6I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_ap6MmlOc2yU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_RnzVPV79IwM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_CjC2YokAYNM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_fvLCqLHC2gg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_fRnyS-6OCAI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_OWU6LzCOOSA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile__OvnbPSQ9ns.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_UXH8WmphVWQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2018_06_30/tile_CXR3m-7oxhw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2018_06_30/tile_VKlB1vsTo1k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2018_06_30/tile_iLYSjIvoV7c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2018_06_30/tile_g5NW4OyrF_w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2018_06_30/tile_UhcGGXD9HTA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2018_06_30/tile_IaKubeBxk-o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2018_06_30/tile_xlHKnJlfjjI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2018_06_30/tile_c5GJn9lqQlg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2018_06_30/tile_A1fOd0d6vfw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2018_06_30/tile_p8fmD97oUw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2018_06_30/tile_j-0-feR5x-Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2018_06_30/tile_D0Ykpy1Q3dE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2018_06_30/tile_fb49UA89wsE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2018_06_30/tile_VxTjtzLnz5s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2018_06_30/tile_MjZ7ijC2MMQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2018_06_30/tile_tI_DZyIguZw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2018_06_30/tile_2CBafV7FlMU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_0A2gJ29ZmZM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_ZhnhFasyWJQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_Af0yq0hni6I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_Fy_29KcjBV4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_DyawyUvpKNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_gUpSW8_n9tk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_WMfRllKRxBs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_lfAYMhyogzA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_b3D3Gy-xk54.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_414GXSY0u0k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_AfMu6nphNKM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_xxXoMLneFKQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_lO3dSzyBHVc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_da5aF9PdeiY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_7Zvh8BTRDDM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_Ei6ZwffbYpg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_BGo4763ZAe4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_-mIbXBC7-fM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_C55GT80DLy4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_zZUrRAwJHcs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_rfD3ZaPrkvA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_LLfSbGafW2s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_6GEy0NhrfLs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2018_06_30/tile_oC7JciLmHEI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_F7op2ob98HQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_YuT9ce3zbXY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_dy3csCGSuNw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_BgGCZSUVKmE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_S9Z4shzmIsE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_vDGSmkdV7os.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_LpZkF1XEY3o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_vI4LhyMPWeg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_YPV1DLqSPIM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_notmacDKViI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_SyLdYKgXsIo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_x7Kq-k7XyAE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_3Q2vR4iJkm8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_sfCXwt6jEz8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_mSVN8yZI-Uk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_h_aPHCxQxSA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_iAKMUFyZAgE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_fSa4zxMdvmc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_szHgiDfUXTY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_66zEbCqh7LA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_HppyIg2uSME.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_Ry0ghFfa2jo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_Dvi1vUuf3Es.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_uahk6r8D0Wo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_2XHdvyceIE0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_oDQaqWtZuxs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_sdYsbVcgUh8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_yqFpuXzmrnw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_n8iP_OQUSLw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_gXWjVVfGCfk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_EV4_heFBtbs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_iNLBAvgkbTg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_P_yj4kitRUw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_eVz79CwsG7U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_oTzNnOPQcss.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_FJlshPGyBoo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_bvz58Sj6DgU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_xGCy64-7DpE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_d-Gf9Hjsfq8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_PDeI1LBjxz8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_gyMYOk8CKtY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_9YDjRVGgNDU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_W-lHKI0gAro.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_iWvpRfo_44w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_lhMLAAYgmBE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_5KlVTvp3Ud8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_26937ENrox4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_4NozDtFdows.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2018_06_30/tile_pgF_G4DWa8E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_edtHlMNdHlk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_5bJrF7sLxoY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_X2U3Wb9Yt2w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_ZMuqD-gMYds.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_M5R2IAMwhZw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_3At_utvBFVU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_yD-5IqzlIt4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_zlZZ6C20Xiw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_hrPqtg2pzLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_z5EXKgocvxo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_4EjvsUke-gg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_ZiMtkI2Qi3c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_hk2xZot2PXE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_0WuKT_oH-c4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_zUe6xvSLPZ8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_ra5oBq3ZhqM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_tAS97RqKbsY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_Qmj1dsXHujU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_ThrufapKc8c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_-S4mAuWhX0s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_a2Ceu-awLLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_Qf2T9y2X8NQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_bQpozx7M-lU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_cek61MplDpc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_fTXx7GhMV_M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_cFkjr0PDIQg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_fnUcJei0LlA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_F43y26WmrH4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_z4UdTx58XLg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_SYDTGycWzaI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_hN6DkXzYW2M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_P0LL9oS_HA8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_YBzNNWaiQTw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_gOzae0pJwvs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2018_06_30/tile_4o-b8zJb58o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2018_06_30/tile_6l8nQqpP3Fs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2018_06_30/tile_Fvzd5OeOzH8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2018_06_30/tile_oJJJQx20xus.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2018_06_30/tile_ndb8nuuMFQE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2018_06_30/tile_xyE-tv1vbpg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2018_06_30/tile_fJmNd-vIC8I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2018_06_30/tile_uyjeDqZaf00.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2018_06_30/tile_ohY41G3cspQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2018_06_30/tile_8bxtBmYoeig.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2018_06_30/tile_r3cl8mRaJw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2018_06_30/tile_aOIVRthWYlM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2018_06_30/tile_ca5H2wWNgOk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2018_06_30/tile_7IbjBqHuIng.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2018_06_30/tile_sYxHTLvk0ZQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_ldBb0Tr1ZwM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_rFvCO4Ah9no.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_5e0MejX9ZxU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_DBuHBh-X59s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_jgxruYmBwN8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_r8fBOaQdJ8E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_qGc0-6zuNLc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_Vdad6TSu3pw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_VRnX6WYY5aY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_FNTgkpAGzbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_2lmC2Q0_NGI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_31fe1gaUUJs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_oBoJG9S-IuU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_Qs4Ftn6U57k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_DzrfLr9c23M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_vQXO29VvamQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_YcYIvV8wU2o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_-lw3NKgt5CE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_eZ0UI9WkUa4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_GE7bCP4ZofQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_Dv6dcOWnI2s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_oWuYRxYPWts.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_9czOQj56LgE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_1jb0MV6ft44.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_GvjN6ss4IjM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_l2Ijk61lArY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_9_oKvfHROyI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_8prFLjh2fCA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_dvw7RgB7zV8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_oA1IXzGzBwU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_NmTWC35JRWA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2018_06_30/tile_EMjG3YF7g-U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_-PV0lk8Wcy0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_BbdgT1OZ4xA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_OebMUMSKp58.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_xAHpfDS_tcY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_Zz62K_GjAEg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_9P8rv_pKqQo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_CiL8thFOPLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_SWbvqMiMruQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_qOmIXpDy4tI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_GalsFR1ogcE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_Xd4BfprZwWo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_KQNKVCrMrew.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_ziyY0HB_UIk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_jOY4cOkLlOI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_Zo-kK8qGZ_A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_At-ZNnlNAdc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2018_06_30/tile_bo1ZdxoCCwQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_VPcv0zeEUUY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_Qgqg0VZsvoo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_l6o10K2govA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_2fJYus8UkwQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_-ElEld5rt5E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_iZkKS0q50w4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_gS5D6klKAkE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_EtmfDSzhAw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_hqTAUj8QqPw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_88jQlqXOVf8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_mgmMDaDtiUQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_nDRZTLCQqvI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_6ViRnrkFi_Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_4bKe6c_tbQo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_RjKOUXqeAsU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_cfa0maP-1FE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_mcPmzf75ToE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_gUx9RYKOj8w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_p_4EDz0m5iU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile__iS3cAV0l1Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_F66qhzaKdUg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_aIt-UR2UqDE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_0chzaXGxohY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_xDRoay6WAbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_T-EVYoBtsvk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_1_M0gDok2-0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_GcKlG8oRV6o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_6RRVAfi_A6Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_3SoNvBqku9E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_iDsfCsFq_II.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_MpAXOG-IkOs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_IdsknuIsqQE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_YcFMiBDiD60.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_6pQK9PWLHRw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_BEpPJrteTbg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_Zw2iwVxp9ok.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_jDOBX14ESqY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_YBUOyhJE9C0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_OC2WfdcNv7I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_atQiFq44XNY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_dFh6u3eGS14.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_EbYnhHZcsoM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_HEYB6rB4afs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_ufajWPtQrts.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_qrp1SrVUIic.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_sCoH8x5AbKI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_mCqEfn-4q4A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_PqI_YrPNSWE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2018_06_30/tile_m6dYq8wGj5U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_bIYrX_YAxwk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_Hu5c9sSMr3A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_vrjZGrwBCA0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_unQMAvozNRk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_0xI9fVduyZE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_wzA4WQgAvrE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_Y8ImHQMAcYA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_58pnWEGSKt8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_yY2x2fammoM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_8nVOcGNu2IA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_RlsKT6Jf6wM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_pq8UH-HEOQg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_VqOmW5kz1W0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_PKm8xyhmIZc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_b_WBA1b7eZA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_FeeFcJAZbYc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_fWsQe-ZeEe8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_UPxsMf5HJWI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_SVIOaY507UM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_QM9EXlxbeBA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_a9oLR3z8Vm0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_R84NkdXV2Z0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_nNA6Pwk-M3U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_jxm7UNj7-Rc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_rc_yujmCW-U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_2OIs0gqeq1k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_Gex4QzSIKy0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_Hc-UpwT69ws.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_gVc0TpxZv7E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_MISuRqfSgeo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_fvrUa3e43x4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_I5fa0cA523A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_JQxlFUS4NfQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_vdnHmna7Ifo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_NmJNE6dWWos.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_xHWoK3gkHe8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_u8FFj7cZHIQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_QWcgi3AtUuU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_4kQAI2-RaC4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_ROOd-NnESME.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_R4y_itQ4hd4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_VFjB7PY-h3w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_MFmAF07BBrc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_F14sWKylN6c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_qUphd3XFpW0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2018_06_30/tile_QtvRPAbpGLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2018_06_30/tile_6RNE_2tn9Z0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2018_06_30/tile_cI_89vaQokA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2018_06_30/tile_5sFxLBA_Iqk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2018_06_30/tile_SrA3hNKGZ64.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2018_06_30/tile_Dd4GpvS8Ge0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2018_06_30/tile_YNgafwyhidk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2018_06_30/tile_t8LbcNrkWrE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2018_06_30/tile_SCFPnVfUql8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2018_06_30/tile_xVbV88yWUhE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa34_2018_06_30/tile_OAmzV52YF2w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa34_2018_06_30/tile_20SedX5wtg8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2018_06_30/tile_8AHkyfOa04w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2018_06_30/tile_25mk9l1sSUU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2018_06_30/tile_ezNXcekn93E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2018_06_30/tile_xV-T0Fy3KFU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2018_06_30/tile_zcYd-KLAVFM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_PGmIEWWtvYg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_lrb8RyXXxUs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_crBc3qKvc_c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_pC6GZFISPXI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_jxnI9CDibDk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_HxMnqi_eHyc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_HwWmPLz2-Vc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_U1sP7RWC1gA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_oHUY41RqYrU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_8qViUzHeLCw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_MuJoJ-9MANU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_-Cuet1TxnNo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_FrFxz3X-7Mo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_AWio5XBXSAc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_9dX3Ngf4Z-4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_gzcQRuzEqLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_0J1CoSS6iPg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_scIfdKTURXA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_ap0j3H6RE1g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile__oLuYSkg12A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_zOGOLNWhXco.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_x73vJhmPBaI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_4Wt4RqUDYv8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_3DxVMYeRslw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_bJ2k97ym_Kw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_yIGOSSIzoMg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_Ig2W_gviuTI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_1YAFw6W52-E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_TF4q8zPmRRc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_mJYLHk7hDEc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_2BM8UBy62go.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_THLy2cMWTJE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_F68y1CtJLsg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_YevJTKbTzTw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_X7Vdb4oIr2Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_xaFmlC3lb-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_DoySuYmtk-w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_KYDEwg89lH0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_vy7ZbKCC8Lk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_W2vgIrSqXIA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_0C3XV_w3Hcc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_U8Xo8TRr30Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_H-lLEvnmoLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_7-krjk7IdVY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_Fsy2GZyiqT8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_Wy2x2VSavKU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_ZxrWWCThOi4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2018_06_30/tile_GSz_zarI5kI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_bvvyF6CZ3v0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_YJmq-27yWK4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_N77B5UsvXQI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_0qjzxzBybu8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_Cfv04geebOc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_wea4hDdk5sM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_oaUjV8M2qVY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_yzO245pSh50.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_YxKi3PRdpE8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_98ZSThIgVNw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_UbaNaWeQQhY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_uys4W-tzfws.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_-6WIgLec_Aw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_uZXb1B-6-Pc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_zaqnXBBelc4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_r9EbIgjkq3Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_pdkAbSnbGXM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2018_06_30/tile_V4S8RzrAoRM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa7c_2018_06_30/tile_8Eo7jYPCsEE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2018_06_30/tile_RxMvIrdiSa4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2018_06_30/tile_eJzTsOJZsUg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2018_06_30/tile_cOL6HYvY8VA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2018_06_30/tile_rMqJeLTuHRE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2018_06_30/tile_nfD2LYI6nYo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2018_06_30/tile_Hjd3hvqGQ1s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2018_06_30/tile_Z37cKCmA-cM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2018_06_30/tile_cFdPOG9KpSE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2018_06_30/tile_GdNxjPqD76I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2018_06_30/tile_o4dsAI5yWBg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2018_06_30/tile_F6QKTYMZ3wI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2018_06_30/tile_5tWtjSZGz_k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2018_06_30/tile_gaHpMg_bTWU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_878QAjksX9k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_5AGXayBwZCM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_95lAfaJ_1o4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_RnIje33rRLY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_iFHrjLE08Vg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_xYnut80Y-zE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_HdPOmocD19w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_UlfQI_P7vco.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_G2nePRpiK6Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_jVMuoz3FmBA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile__lHwd88WlAk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_uA6JC01J-II.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_n6WL2zzxprU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_e7OyCSqpV-c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_MeDOXFSvFNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_uptv8iGJBi4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_PiL6Joun_-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_j1y0l7fP-Sk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_NYfDvlcPv0c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_KhF1V8P1Z20.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_wcCkxNyBXr4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2018_06_30/tile_MJ1U3NpVqR8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_nPkNNFAZKLM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_9v8P-w1N6_g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_LCXbkT5Dbv4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_rO4GFLbHNqk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_SjJwsggAeY4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_HwzR7aY6WIo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_zikBG5UPINA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_-0lEnLgfrGU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_VyEdJOyGwNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_JyXtw1WXzu4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile__qWrETqMx48.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_hv51HgzPQjc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_DWzHgPsDu_c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_CRQSl5Iz7Rk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_TcomU2q_loQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_bSSp3yNUssU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_EtzjtiiE4ZY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_siUk0QP_z8g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_otJdZNEr8X8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_m_g0A3UNuGY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_rrzf2r3R5Js.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_os-FTpdD5sc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_lAXVJ1CZ8PE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_f6mhLCvACJk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_hGDdIkHOdLU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_MUTAkRKvdHI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_EdNson2UbQc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_GTMv-a6-QQI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_Me2UovXVGlw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_kVYBvRC38zs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_FUN5e6yclBM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_whdVajibyDI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_igkH8p44L3U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_obdfPw2Hu3s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_6ITL_yjNmww.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_TvXw-dGPgqA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_kjMdbwQjorw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_bfkzc-Hocc0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_xU55vI4txgU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_iELDzowUH7M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_XhrqU2iikHQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_BiuWipHcYKQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2018_06_30/tile_v0HPhTzrt20.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_9z5qs7azJEg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_UeFS52yPtVE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_-UHrdUFyL5Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_jU67INtOoPE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_ibeuqkRRCDI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile__5iKyN8m6Og.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_dvywPun0dsA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_VWJc22h_hBQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_8_DRwoUsU9Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_NIVD-Z4zMU8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_w4XApkuShgs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_1yMS-7nDS5s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_lhR5sXnbzRk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_xhU82Q9ccg8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_GlSCj7IFwnY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_P_B1sisDlRY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_eXoQAjVcOYQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_vW2MJxoFgMk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_Sq3-rJUxsNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_xQ-44Zay7aA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_xDIcGmh6HVg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_CWZrLuhZaro.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_I67YZsLB-kw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_s9WQ6imuVJI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_Wdo7UBfds04.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_kYInGFjnjWA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_NGJz-evoHWQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_I_aWqMwULzI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2018_06_30/tile_IN36N6xwaq0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2018_06_30/tile_I1OMvjAHoc8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2018_06_30/tile_MVyILXNgnas.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2018_06_30/tile_dPY0-2oQmtU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2018_06_30/tile_LBRA9z5hNF4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2018_06_30/tile_XacG0jFDyH8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2018_06_30/tile_5WXILtuOQbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2018_06_30/tile_uedmM83ZqRs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2018_06_30/tile_H42ZeH4gFtI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2018_06_30/tile_4PJBpAUrAqw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2018_06_30/tile_y4YR_QsaTcQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2018_06_30/tile_72OrTmz8LwY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2018_06_30/tile_45CwXin5MhE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2018_06_30/tile_rxgV4aC8GWg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2018_06_30/tile_hn2htg6Y1KQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2018_06_30/tile_R6kRjlEAS2Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2018_06_30/tile_PZ7IGn7F_pE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_ldBb0Tr1ZwM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_rFvCO4Ah9no.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_5e0MejX9ZxU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_DBuHBh-X59s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_jgxruYmBwN8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_r8fBOaQdJ8E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_qGc0-6zuNLc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_Vdad6TSu3pw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_VRnX6WYY5aY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_FNTgkpAGzbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_2lmC2Q0_NGI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_31fe1gaUUJs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_oBoJG9S-IuU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_Qs4Ftn6U57k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_DzrfLr9c23M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_vQXO29VvamQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_YcYIvV8wU2o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_-lw3NKgt5CE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_eZ0UI9WkUa4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_GE7bCP4ZofQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_Dv6dcOWnI2s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_oWuYRxYPWts.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_9czOQj56LgE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_1jb0MV6ft44.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_GvjN6ss4IjM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_l2Ijk61lArY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_9_oKvfHROyI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_8prFLjh2fCA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_dvw7RgB7zV8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_oA1IXzGzBwU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_NmTWC35JRWA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2019_06_30/tile_EMjG3YF7g-U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_F7op2ob98HQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_YuT9ce3zbXY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_dy3csCGSuNw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_BgGCZSUVKmE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_S9Z4shzmIsE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_vDGSmkdV7os.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_LpZkF1XEY3o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_vI4LhyMPWeg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_YPV1DLqSPIM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_notmacDKViI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_SyLdYKgXsIo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_x7Kq-k7XyAE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_3Q2vR4iJkm8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_sfCXwt6jEz8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_mSVN8yZI-Uk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_h_aPHCxQxSA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_iAKMUFyZAgE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_fSa4zxMdvmc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_szHgiDfUXTY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_66zEbCqh7LA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_HppyIg2uSME.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_Ry0ghFfa2jo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_Dvi1vUuf3Es.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_uahk6r8D0Wo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_2XHdvyceIE0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_oDQaqWtZuxs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_sdYsbVcgUh8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_yqFpuXzmrnw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_n8iP_OQUSLw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_gXWjVVfGCfk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_EV4_heFBtbs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_iNLBAvgkbTg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_P_yj4kitRUw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_eVz79CwsG7U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_oTzNnOPQcss.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_FJlshPGyBoo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_bvz58Sj6DgU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_xGCy64-7DpE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_d-Gf9Hjsfq8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_PDeI1LBjxz8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_gyMYOk8CKtY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_9YDjRVGgNDU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_W-lHKI0gAro.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_iWvpRfo_44w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_lhMLAAYgmBE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_5KlVTvp3Ud8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_26937ENrox4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_4NozDtFdows.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2019_06_30/tile_pgF_G4DWa8E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_0A2gJ29ZmZM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_ZhnhFasyWJQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_Af0yq0hni6I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_Fy_29KcjBV4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_DyawyUvpKNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_gUpSW8_n9tk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_WMfRllKRxBs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_lfAYMhyogzA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_b3D3Gy-xk54.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_414GXSY0u0k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_AfMu6nphNKM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_xxXoMLneFKQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_lO3dSzyBHVc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_da5aF9PdeiY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_7Zvh8BTRDDM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_Ei6ZwffbYpg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_BGo4763ZAe4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_-mIbXBC7-fM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_C55GT80DLy4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_zZUrRAwJHcs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_rfD3ZaPrkvA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_LLfSbGafW2s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_6GEy0NhrfLs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2019_06_30/tile_oC7JciLmHEI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9ec_2019_06_30/tile_o55i51eydaQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_edtHlMNdHlk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_5bJrF7sLxoY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_X2U3Wb9Yt2w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_ZMuqD-gMYds.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_M5R2IAMwhZw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_3At_utvBFVU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_yD-5IqzlIt4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_zlZZ6C20Xiw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_hrPqtg2pzLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_z5EXKgocvxo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_4EjvsUke-gg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_ZiMtkI2Qi3c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_hk2xZot2PXE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_0WuKT_oH-c4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_zUe6xvSLPZ8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_ra5oBq3ZhqM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_tAS97RqKbsY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_Qmj1dsXHujU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_ThrufapKc8c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_-S4mAuWhX0s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_a2Ceu-awLLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_Qf2T9y2X8NQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_bQpozx7M-lU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_cek61MplDpc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_fTXx7GhMV_M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_cFkjr0PDIQg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_fnUcJei0LlA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_F43y26WmrH4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_z4UdTx58XLg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_SYDTGycWzaI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_hN6DkXzYW2M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_P0LL9oS_HA8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_YBzNNWaiQTw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_gOzae0pJwvs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2019_06_30/tile_4o-b8zJb58o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_ncdQ401nuQM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_tnaJo4bzUSI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_3PP94IvU6LU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_OGCSC_GRupA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_xxz7CbfxxI0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_ovl1QE5MZxM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_-v3t1LPEWFE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_2UU7iRXs0pw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_Kaej5n7IJdI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_iM8BvsuXYgo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_bdveB5gKAR8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_Rr4IwM4hAR4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_dr5N5SqIUdc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_4WsINMFjQyA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_OuCHhfHiIiM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_m9WKjXykcrw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_za6lbJmtBNA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_zmVokDgk-Qc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_BvnC-jJY7wY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_n8T27F4DPvU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_t3tWUMfnM1s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile___2Ly15PD-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_N9fYMYCIDXM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_8u4vsdT3ezc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_zJgy7xJ6s14.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_XwIpcN0I42o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_KYwaBtt8bmc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_CgaoNS4tIPY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_BhG1_EGrxp0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_Dmbab34e-DM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_LoB42QzAKR4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_k_rYgCSlHoc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_Sm-4CiwChiU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_YdOJsuQ3NTE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_Wr5S4gpj6PI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_2sNXli1EiBM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_X4FrkiIFSFw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_oU9nU8lnzeU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_bu142fanRU4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_320NPFa_c6I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_ap6MmlOc2yU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_RnzVPV79IwM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_CjC2YokAYNM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_fvLCqLHC2gg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_fRnyS-6OCAI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_OWU6LzCOOSA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile__OvnbPSQ9ns.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_UXH8WmphVWQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2019_06_30/tile_CXR3m-7oxhw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2019_06_30/tile_VKlB1vsTo1k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2019_06_30/tile_iLYSjIvoV7c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2019_06_30/tile_g5NW4OyrF_w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2019_06_30/tile_UhcGGXD9HTA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2019_06_30/tile_IaKubeBxk-o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2019_06_30/tile_xlHKnJlfjjI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2019_06_30/tile_c5GJn9lqQlg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2019_06_30/tile_A1fOd0d6vfw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2019_06_30/tile_p8fmD97oUw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2019_06_30/tile_j-0-feR5x-Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2019_06_30/tile_D0Ykpy1Q3dE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2019_06_30/tile_fb49UA89wsE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2019_06_30/tile_VxTjtzLnz5s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2019_06_30/tile_MjZ7ijC2MMQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2019_06_30/tile_tI_DZyIguZw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2019_06_30/tile_2CBafV7FlMU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_-PV0lk8Wcy0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_BbdgT1OZ4xA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_OebMUMSKp58.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_xAHpfDS_tcY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_Zz62K_GjAEg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_9P8rv_pKqQo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_CiL8thFOPLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_SWbvqMiMruQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_qOmIXpDy4tI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_GalsFR1ogcE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_Xd4BfprZwWo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_KQNKVCrMrew.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_ziyY0HB_UIk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_jOY4cOkLlOI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_Zo-kK8qGZ_A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_At-ZNnlNAdc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2019_06_30/tile_bo1ZdxoCCwQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2019_06_30/tile_6l8nQqpP3Fs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2019_06_30/tile_Fvzd5OeOzH8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2019_06_30/tile_oJJJQx20xus.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2019_06_30/tile_ndb8nuuMFQE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2019_06_30/tile_xyE-tv1vbpg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2019_06_30/tile_fJmNd-vIC8I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2019_06_30/tile_uyjeDqZaf00.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2019_06_30/tile_ohY41G3cspQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2019_06_30/tile_8bxtBmYoeig.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2019_06_30/tile_r3cl8mRaJw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2019_06_30/tile_aOIVRthWYlM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2019_06_30/tile_ca5H2wWNgOk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2019_06_30/tile_7IbjBqHuIng.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2019_06_30/tile_sYxHTLvk0ZQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2019_06_30/tile_8AHkyfOa04w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2019_06_30/tile_25mk9l1sSUU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2019_06_30/tile_ezNXcekn93E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2019_06_30/tile_xV-T0Fy3KFU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2019_06_30/tile_zcYd-KLAVFM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_VPcv0zeEUUY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_Qgqg0VZsvoo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_l6o10K2govA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_2fJYus8UkwQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_-ElEld5rt5E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_iZkKS0q50w4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_gS5D6klKAkE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_EtmfDSzhAw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_hqTAUj8QqPw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_88jQlqXOVf8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_mgmMDaDtiUQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_nDRZTLCQqvI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_6ViRnrkFi_Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_4bKe6c_tbQo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_RjKOUXqeAsU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_cfa0maP-1FE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_mcPmzf75ToE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_gUx9RYKOj8w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_p_4EDz0m5iU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile__iS3cAV0l1Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_F66qhzaKdUg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_aIt-UR2UqDE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_0chzaXGxohY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_xDRoay6WAbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_T-EVYoBtsvk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_1_M0gDok2-0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_GcKlG8oRV6o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_6RRVAfi_A6Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_3SoNvBqku9E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_iDsfCsFq_II.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_MpAXOG-IkOs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_IdsknuIsqQE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_YcFMiBDiD60.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_6pQK9PWLHRw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_BEpPJrteTbg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_Zw2iwVxp9ok.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_jDOBX14ESqY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_YBUOyhJE9C0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_OC2WfdcNv7I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_atQiFq44XNY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_dFh6u3eGS14.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_EbYnhHZcsoM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_HEYB6rB4afs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_ufajWPtQrts.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_qrp1SrVUIic.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_sCoH8x5AbKI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_mCqEfn-4q4A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_PqI_YrPNSWE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2019_06_30/tile_m6dYq8wGj5U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_nPkNNFAZKLM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_9v8P-w1N6_g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_LCXbkT5Dbv4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_rO4GFLbHNqk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_SjJwsggAeY4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_HwzR7aY6WIo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_zikBG5UPINA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_-0lEnLgfrGU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_VyEdJOyGwNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_JyXtw1WXzu4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile__qWrETqMx48.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_hv51HgzPQjc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_DWzHgPsDu_c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_CRQSl5Iz7Rk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_TcomU2q_loQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_bSSp3yNUssU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_EtzjtiiE4ZY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_siUk0QP_z8g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_otJdZNEr8X8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_m_g0A3UNuGY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_rrzf2r3R5Js.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_os-FTpdD5sc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_lAXVJ1CZ8PE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_f6mhLCvACJk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_hGDdIkHOdLU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_MUTAkRKvdHI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_EdNson2UbQc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_GTMv-a6-QQI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_Me2UovXVGlw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_kVYBvRC38zs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_FUN5e6yclBM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_whdVajibyDI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_igkH8p44L3U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_obdfPw2Hu3s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_6ITL_yjNmww.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_TvXw-dGPgqA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_kjMdbwQjorw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_bfkzc-Hocc0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_xU55vI4txgU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_iELDzowUH7M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_XhrqU2iikHQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_BiuWipHcYKQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2019_06_30/tile_v0HPhTzrt20.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2019_06_30/tile_I1OMvjAHoc8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2019_06_30/tile_MVyILXNgnas.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2019_06_30/tile_dPY0-2oQmtU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2019_06_30/tile_LBRA9z5hNF4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2019_06_30/tile_XacG0jFDyH8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2019_06_30/tile_5WXILtuOQbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2019_06_30/tile_uedmM83ZqRs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2019_06_30/tile_H42ZeH4gFtI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2019_06_30/tile_4PJBpAUrAqw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2019_06_30/tile_y4YR_QsaTcQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2019_06_30/tile_72OrTmz8LwY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2019_06_30/tile_45CwXin5MhE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2019_06_30/tile_rxgV4aC8GWg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2019_06_30/tile_hn2htg6Y1KQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2019_06_30/tile_R6kRjlEAS2Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2019_06_30/tile_PZ7IGn7F_pE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_bvvyF6CZ3v0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_YJmq-27yWK4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_N77B5UsvXQI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_0qjzxzBybu8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_Cfv04geebOc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_wea4hDdk5sM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_oaUjV8M2qVY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_yzO245pSh50.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_YxKi3PRdpE8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_98ZSThIgVNw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_UbaNaWeQQhY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_uys4W-tzfws.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_-6WIgLec_Aw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_uZXb1B-6-Pc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_zaqnXBBelc4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_r9EbIgjkq3Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_pdkAbSnbGXM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2019_06_30/tile_V4S8RzrAoRM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2019_06_30/tile_rMqJeLTuHRE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2019_06_30/tile_nfD2LYI6nYo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2019_06_30/tile_Hjd3hvqGQ1s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2019_06_30/tile_Z37cKCmA-cM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2019_06_30/tile_cFdPOG9KpSE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2019_06_30/tile_GdNxjPqD76I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2019_06_30/tile_o4dsAI5yWBg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2019_06_30/tile_F6QKTYMZ3wI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2019_06_30/tile_5tWtjSZGz_k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2019_06_30/tile_gaHpMg_bTWU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2019_06_30/tile_RxMvIrdiSa4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2019_06_30/tile_eJzTsOJZsUg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2019_06_30/tile_cOL6HYvY8VA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa7c_2019_06_30/tile_8Eo7jYPCsEE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa34_2019_06_30/tile_OAmzV52YF2w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa34_2019_06_30/tile_20SedX5wtg8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_878QAjksX9k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_5AGXayBwZCM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_95lAfaJ_1o4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_RnIje33rRLY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_iFHrjLE08Vg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_xYnut80Y-zE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_HdPOmocD19w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_UlfQI_P7vco.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_G2nePRpiK6Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_jVMuoz3FmBA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile__lHwd88WlAk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_uA6JC01J-II.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_n6WL2zzxprU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_e7OyCSqpV-c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_MeDOXFSvFNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_uptv8iGJBi4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_PiL6Joun_-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_j1y0l7fP-Sk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_NYfDvlcPv0c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_KhF1V8P1Z20.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_wcCkxNyBXr4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2019_06_30/tile_MJ1U3NpVqR8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_bIYrX_YAxwk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_Hu5c9sSMr3A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_vrjZGrwBCA0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_unQMAvozNRk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_0xI9fVduyZE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_wzA4WQgAvrE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_Y8ImHQMAcYA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_58pnWEGSKt8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_yY2x2fammoM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_8nVOcGNu2IA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_RlsKT6Jf6wM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_pq8UH-HEOQg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_VqOmW5kz1W0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_PKm8xyhmIZc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_b_WBA1b7eZA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_FeeFcJAZbYc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_fWsQe-ZeEe8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_UPxsMf5HJWI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_SVIOaY507UM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_QM9EXlxbeBA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_a9oLR3z8Vm0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_R84NkdXV2Z0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_nNA6Pwk-M3U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_jxm7UNj7-Rc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_rc_yujmCW-U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_2OIs0gqeq1k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_Gex4QzSIKy0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_Hc-UpwT69ws.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_gVc0TpxZv7E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_MISuRqfSgeo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_fvrUa3e43x4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_I5fa0cA523A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_JQxlFUS4NfQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_vdnHmna7Ifo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_NmJNE6dWWos.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_xHWoK3gkHe8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_u8FFj7cZHIQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_QWcgi3AtUuU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_4kQAI2-RaC4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_ROOd-NnESME.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_R4y_itQ4hd4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_VFjB7PY-h3w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_MFmAF07BBrc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_F14sWKylN6c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_qUphd3XFpW0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2019_06_30/tile_QtvRPAbpGLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_PGmIEWWtvYg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_lrb8RyXXxUs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_crBc3qKvc_c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_pC6GZFISPXI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_jxnI9CDibDk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_HxMnqi_eHyc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_HwWmPLz2-Vc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_U1sP7RWC1gA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_oHUY41RqYrU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_8qViUzHeLCw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_MuJoJ-9MANU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_-Cuet1TxnNo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_FrFxz3X-7Mo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_AWio5XBXSAc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_9dX3Ngf4Z-4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_gzcQRuzEqLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_0J1CoSS6iPg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_scIfdKTURXA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_ap0j3H6RE1g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile__oLuYSkg12A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_zOGOLNWhXco.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_x73vJhmPBaI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_4Wt4RqUDYv8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_3DxVMYeRslw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_bJ2k97ym_Kw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_yIGOSSIzoMg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_Ig2W_gviuTI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_1YAFw6W52-E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_TF4q8zPmRRc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_mJYLHk7hDEc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_2BM8UBy62go.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_THLy2cMWTJE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_F68y1CtJLsg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_YevJTKbTzTw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_X7Vdb4oIr2Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_xaFmlC3lb-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_DoySuYmtk-w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_KYDEwg89lH0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_vy7ZbKCC8Lk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_W2vgIrSqXIA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_0C3XV_w3Hcc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_U8Xo8TRr30Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_H-lLEvnmoLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_7-krjk7IdVY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_Fsy2GZyiqT8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_Wy2x2VSavKU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_ZxrWWCThOi4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2019_06_30/tile_GSz_zarI5kI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2019_06_30/tile_6RNE_2tn9Z0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2019_06_30/tile_cI_89vaQokA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2019_06_30/tile_5sFxLBA_Iqk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2019_06_30/tile_SrA3hNKGZ64.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2019_06_30/tile_Dd4GpvS8Ge0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2019_06_30/tile_YNgafwyhidk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2019_06_30/tile_t8LbcNrkWrE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2019_06_30/tile_SCFPnVfUql8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2019_06_30/tile_xVbV88yWUhE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_9z5qs7azJEg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_UeFS52yPtVE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_-UHrdUFyL5Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_jU67INtOoPE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_ibeuqkRRCDI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile__5iKyN8m6Og.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_dvywPun0dsA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_VWJc22h_hBQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_8_DRwoUsU9Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_NIVD-Z4zMU8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_w4XApkuShgs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_1yMS-7nDS5s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_lhR5sXnbzRk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_xhU82Q9ccg8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_GlSCj7IFwnY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_P_B1sisDlRY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_eXoQAjVcOYQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_vW2MJxoFgMk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_Sq3-rJUxsNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_xQ-44Zay7aA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_xDIcGmh6HVg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_CWZrLuhZaro.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_I67YZsLB-kw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_s9WQ6imuVJI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_Wdo7UBfds04.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_kYInGFjnjWA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_NGJz-evoHWQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_I_aWqMwULzI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2019_06_30/tile_IN36N6xwaq0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_edtHlMNdHlk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_5bJrF7sLxoY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_X2U3Wb9Yt2w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_ZMuqD-gMYds.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_M5R2IAMwhZw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_3At_utvBFVU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_yD-5IqzlIt4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_zlZZ6C20Xiw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_hrPqtg2pzLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_z5EXKgocvxo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_4EjvsUke-gg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_ZiMtkI2Qi3c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_hk2xZot2PXE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_0WuKT_oH-c4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_zUe6xvSLPZ8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_ra5oBq3ZhqM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_tAS97RqKbsY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_Qmj1dsXHujU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_ThrufapKc8c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_-S4mAuWhX0s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_a2Ceu-awLLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_Qf2T9y2X8NQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_bQpozx7M-lU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_cek61MplDpc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_fTXx7GhMV_M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_cFkjr0PDIQg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_fnUcJei0LlA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_F43y26WmrH4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_z4UdTx58XLg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_SYDTGycWzaI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_hN6DkXzYW2M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_P0LL9oS_HA8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_YBzNNWaiQTw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_gOzae0pJwvs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2020_06_30/tile_4o-b8zJb58o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_ldBb0Tr1ZwM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_rFvCO4Ah9no.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_5e0MejX9ZxU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_DBuHBh-X59s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_jgxruYmBwN8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_r8fBOaQdJ8E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_qGc0-6zuNLc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_Vdad6TSu3pw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_VRnX6WYY5aY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_FNTgkpAGzbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_2lmC2Q0_NGI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_31fe1gaUUJs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_oBoJG9S-IuU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_Qs4Ftn6U57k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_DzrfLr9c23M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_vQXO29VvamQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_YcYIvV8wU2o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_-lw3NKgt5CE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_eZ0UI9WkUa4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_GE7bCP4ZofQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_Dv6dcOWnI2s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_oWuYRxYPWts.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_9czOQj56LgE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_1jb0MV6ft44.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_GvjN6ss4IjM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_l2Ijk61lArY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_9_oKvfHROyI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_8prFLjh2fCA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_dvw7RgB7zV8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_oA1IXzGzBwU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_NmTWC35JRWA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2020_06_30/tile_EMjG3YF7g-U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_F7op2ob98HQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_YuT9ce3zbXY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_dy3csCGSuNw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_BgGCZSUVKmE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_S9Z4shzmIsE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_vDGSmkdV7os.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_LpZkF1XEY3o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_vI4LhyMPWeg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_YPV1DLqSPIM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_notmacDKViI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_SyLdYKgXsIo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_x7Kq-k7XyAE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_3Q2vR4iJkm8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_sfCXwt6jEz8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_mSVN8yZI-Uk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_h_aPHCxQxSA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_iAKMUFyZAgE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_fSa4zxMdvmc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_szHgiDfUXTY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_66zEbCqh7LA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_HppyIg2uSME.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_Ry0ghFfa2jo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_Dvi1vUuf3Es.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_uahk6r8D0Wo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_2XHdvyceIE0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_oDQaqWtZuxs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_sdYsbVcgUh8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_yqFpuXzmrnw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_n8iP_OQUSLw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_gXWjVVfGCfk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_EV4_heFBtbs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_iNLBAvgkbTg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_P_yj4kitRUw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_eVz79CwsG7U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_oTzNnOPQcss.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_FJlshPGyBoo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_bvz58Sj6DgU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_xGCy64-7DpE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_d-Gf9Hjsfq8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_PDeI1LBjxz8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_gyMYOk8CKtY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_9YDjRVGgNDU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_W-lHKI0gAro.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_iWvpRfo_44w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_lhMLAAYgmBE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_5KlVTvp3Ud8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_26937ENrox4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_4NozDtFdows.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2020_06_30/tile_pgF_G4DWa8E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_-PV0lk8Wcy0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_BbdgT1OZ4xA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_OebMUMSKp58.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_xAHpfDS_tcY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_Zz62K_GjAEg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_9P8rv_pKqQo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_CiL8thFOPLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_SWbvqMiMruQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_qOmIXpDy4tI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_GalsFR1ogcE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_Xd4BfprZwWo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_KQNKVCrMrew.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_ziyY0HB_UIk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_jOY4cOkLlOI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_Zo-kK8qGZ_A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_At-ZNnlNAdc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2020_06_30/tile_bo1ZdxoCCwQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2020_06_30/tile_VKlB1vsTo1k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2020_06_30/tile_iLYSjIvoV7c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2020_06_30/tile_g5NW4OyrF_w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2020_06_30/tile_UhcGGXD9HTA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2020_06_30/tile_IaKubeBxk-o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2020_06_30/tile_xlHKnJlfjjI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2020_06_30/tile_c5GJn9lqQlg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2020_06_30/tile_A1fOd0d6vfw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2020_06_30/tile_p8fmD97oUw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2020_06_30/tile_j-0-feR5x-Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2020_06_30/tile_D0Ykpy1Q3dE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2020_06_30/tile_fb49UA89wsE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2020_06_30/tile_VxTjtzLnz5s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2020_06_30/tile_MjZ7ijC2MMQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2020_06_30/tile_tI_DZyIguZw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2020_06_30/tile_2CBafV7FlMU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_0A2gJ29ZmZM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_ZhnhFasyWJQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_Af0yq0hni6I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_Fy_29KcjBV4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_DyawyUvpKNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_gUpSW8_n9tk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_WMfRllKRxBs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_lfAYMhyogzA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_b3D3Gy-xk54.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_414GXSY0u0k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_AfMu6nphNKM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_xxXoMLneFKQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_lO3dSzyBHVc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_da5aF9PdeiY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_7Zvh8BTRDDM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_Ei6ZwffbYpg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_BGo4763ZAe4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_-mIbXBC7-fM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_C55GT80DLy4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_zZUrRAwJHcs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_rfD3ZaPrkvA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_LLfSbGafW2s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_6GEy0NhrfLs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2020_06_30/tile_oC7JciLmHEI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9ec_2020_06_30/tile_o55i51eydaQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_ncdQ401nuQM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_tnaJo4bzUSI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_3PP94IvU6LU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_OGCSC_GRupA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_xxz7CbfxxI0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_ovl1QE5MZxM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_-v3t1LPEWFE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_2UU7iRXs0pw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_Kaej5n7IJdI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_iM8BvsuXYgo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_bdveB5gKAR8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_Rr4IwM4hAR4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_dr5N5SqIUdc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_4WsINMFjQyA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_OuCHhfHiIiM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_m9WKjXykcrw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_za6lbJmtBNA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_zmVokDgk-Qc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_BvnC-jJY7wY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_n8T27F4DPvU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_t3tWUMfnM1s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile___2Ly15PD-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_N9fYMYCIDXM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_8u4vsdT3ezc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_zJgy7xJ6s14.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_XwIpcN0I42o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_KYwaBtt8bmc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_CgaoNS4tIPY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_BhG1_EGrxp0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_Dmbab34e-DM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_LoB42QzAKR4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_k_rYgCSlHoc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_Sm-4CiwChiU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_YdOJsuQ3NTE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_Wr5S4gpj6PI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_2sNXli1EiBM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_X4FrkiIFSFw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_oU9nU8lnzeU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_bu142fanRU4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_320NPFa_c6I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_ap6MmlOc2yU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_RnzVPV79IwM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_CjC2YokAYNM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_fvLCqLHC2gg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_fRnyS-6OCAI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_OWU6LzCOOSA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile__OvnbPSQ9ns.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_UXH8WmphVWQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2020_06_30/tile_CXR3m-7oxhw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2020_06_30/tile_6l8nQqpP3Fs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2020_06_30/tile_Fvzd5OeOzH8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2020_06_30/tile_oJJJQx20xus.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2020_06_30/tile_ndb8nuuMFQE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2020_06_30/tile_xyE-tv1vbpg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2020_06_30/tile_fJmNd-vIC8I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2020_06_30/tile_uyjeDqZaf00.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2020_06_30/tile_ohY41G3cspQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2020_06_30/tile_8bxtBmYoeig.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2020_06_30/tile_r3cl8mRaJw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2020_06_30/tile_aOIVRthWYlM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2020_06_30/tile_ca5H2wWNgOk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2020_06_30/tile_7IbjBqHuIng.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2020_06_30/tile_sYxHTLvk0ZQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2020_06_30/tile_RxMvIrdiSa4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2020_06_30/tile_eJzTsOJZsUg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2020_06_30/tile_cOL6HYvY8VA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_bIYrX_YAxwk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_Hu5c9sSMr3A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_vrjZGrwBCA0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_unQMAvozNRk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_0xI9fVduyZE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_wzA4WQgAvrE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_Y8ImHQMAcYA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_58pnWEGSKt8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_yY2x2fammoM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_8nVOcGNu2IA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_RlsKT6Jf6wM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_pq8UH-HEOQg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_VqOmW5kz1W0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_PKm8xyhmIZc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_b_WBA1b7eZA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_FeeFcJAZbYc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_fWsQe-ZeEe8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_UPxsMf5HJWI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_SVIOaY507UM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_QM9EXlxbeBA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_a9oLR3z8Vm0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_R84NkdXV2Z0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_nNA6Pwk-M3U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_jxm7UNj7-Rc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_rc_yujmCW-U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_2OIs0gqeq1k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_Gex4QzSIKy0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_Hc-UpwT69ws.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_gVc0TpxZv7E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_MISuRqfSgeo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_fvrUa3e43x4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_I5fa0cA523A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_JQxlFUS4NfQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_vdnHmna7Ifo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_NmJNE6dWWos.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_xHWoK3gkHe8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_u8FFj7cZHIQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_QWcgi3AtUuU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_4kQAI2-RaC4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_ROOd-NnESME.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_R4y_itQ4hd4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_VFjB7PY-h3w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_MFmAF07BBrc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_F14sWKylN6c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_qUphd3XFpW0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2020_06_30/tile_QtvRPAbpGLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2020_06_30/tile_rMqJeLTuHRE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2020_06_30/tile_nfD2LYI6nYo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2020_06_30/tile_Hjd3hvqGQ1s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2020_06_30/tile_Z37cKCmA-cM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2020_06_30/tile_cFdPOG9KpSE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2020_06_30/tile_GdNxjPqD76I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2020_06_30/tile_o4dsAI5yWBg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2020_06_30/tile_F6QKTYMZ3wI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2020_06_30/tile_5tWtjSZGz_k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2020_06_30/tile_gaHpMg_bTWU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_9z5qs7azJEg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_UeFS52yPtVE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_-UHrdUFyL5Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_jU67INtOoPE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_ibeuqkRRCDI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile__5iKyN8m6Og.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_dvywPun0dsA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_VWJc22h_hBQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_8_DRwoUsU9Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_NIVD-Z4zMU8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_w4XApkuShgs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_1yMS-7nDS5s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_lhR5sXnbzRk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_xhU82Q9ccg8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_GlSCj7IFwnY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_P_B1sisDlRY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_eXoQAjVcOYQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_vW2MJxoFgMk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_Sq3-rJUxsNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_xQ-44Zay7aA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_xDIcGmh6HVg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_CWZrLuhZaro.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_I67YZsLB-kw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_s9WQ6imuVJI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_Wdo7UBfds04.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_kYInGFjnjWA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_NGJz-evoHWQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_I_aWqMwULzI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2020_06_30/tile_IN36N6xwaq0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_PGmIEWWtvYg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_lrb8RyXXxUs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_crBc3qKvc_c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_pC6GZFISPXI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_jxnI9CDibDk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_HxMnqi_eHyc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_HwWmPLz2-Vc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_U1sP7RWC1gA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_oHUY41RqYrU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_8qViUzHeLCw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_MuJoJ-9MANU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_-Cuet1TxnNo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_FrFxz3X-7Mo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_AWio5XBXSAc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_9dX3Ngf4Z-4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_gzcQRuzEqLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_0J1CoSS6iPg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_scIfdKTURXA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_ap0j3H6RE1g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile__oLuYSkg12A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_zOGOLNWhXco.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_x73vJhmPBaI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_4Wt4RqUDYv8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_3DxVMYeRslw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_bJ2k97ym_Kw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_yIGOSSIzoMg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_Ig2W_gviuTI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_1YAFw6W52-E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_TF4q8zPmRRc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_mJYLHk7hDEc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_2BM8UBy62go.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_THLy2cMWTJE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_F68y1CtJLsg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_YevJTKbTzTw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_X7Vdb4oIr2Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_xaFmlC3lb-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_DoySuYmtk-w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_KYDEwg89lH0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_vy7ZbKCC8Lk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_W2vgIrSqXIA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_0C3XV_w3Hcc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_U8Xo8TRr30Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_H-lLEvnmoLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_7-krjk7IdVY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_Fsy2GZyiqT8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_Wy2x2VSavKU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_ZxrWWCThOi4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2020_06_30/tile_GSz_zarI5kI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_nPkNNFAZKLM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_9v8P-w1N6_g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_LCXbkT5Dbv4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_rO4GFLbHNqk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_SjJwsggAeY4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_HwzR7aY6WIo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_zikBG5UPINA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_-0lEnLgfrGU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_VyEdJOyGwNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_JyXtw1WXzu4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile__qWrETqMx48.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_hv51HgzPQjc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_DWzHgPsDu_c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_CRQSl5Iz7Rk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_TcomU2q_loQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_bSSp3yNUssU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_EtzjtiiE4ZY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_siUk0QP_z8g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_otJdZNEr8X8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_m_g0A3UNuGY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_rrzf2r3R5Js.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_os-FTpdD5sc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_lAXVJ1CZ8PE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_f6mhLCvACJk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_hGDdIkHOdLU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_MUTAkRKvdHI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_EdNson2UbQc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_GTMv-a6-QQI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_Me2UovXVGlw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_kVYBvRC38zs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_FUN5e6yclBM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_whdVajibyDI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_igkH8p44L3U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_obdfPw2Hu3s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_6ITL_yjNmww.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_TvXw-dGPgqA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_kjMdbwQjorw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_bfkzc-Hocc0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_xU55vI4txgU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_iELDzowUH7M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_XhrqU2iikHQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_BiuWipHcYKQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2020_06_30/tile_v0HPhTzrt20.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_VPcv0zeEUUY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_Qgqg0VZsvoo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_l6o10K2govA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_2fJYus8UkwQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_-ElEld5rt5E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_iZkKS0q50w4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_gS5D6klKAkE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_EtmfDSzhAw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_hqTAUj8QqPw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_88jQlqXOVf8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_mgmMDaDtiUQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_nDRZTLCQqvI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_6ViRnrkFi_Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_4bKe6c_tbQo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_RjKOUXqeAsU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_cfa0maP-1FE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_mcPmzf75ToE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_gUx9RYKOj8w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_p_4EDz0m5iU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile__iS3cAV0l1Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_F66qhzaKdUg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_aIt-UR2UqDE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_0chzaXGxohY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_xDRoay6WAbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_T-EVYoBtsvk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_1_M0gDok2-0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_GcKlG8oRV6o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_6RRVAfi_A6Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_3SoNvBqku9E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_iDsfCsFq_II.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_MpAXOG-IkOs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_IdsknuIsqQE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_YcFMiBDiD60.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_6pQK9PWLHRw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_BEpPJrteTbg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_Zw2iwVxp9ok.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_jDOBX14ESqY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_YBUOyhJE9C0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_OC2WfdcNv7I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_atQiFq44XNY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_dFh6u3eGS14.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_EbYnhHZcsoM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_HEYB6rB4afs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_ufajWPtQrts.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_qrp1SrVUIic.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_sCoH8x5AbKI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_mCqEfn-4q4A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_PqI_YrPNSWE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2020_06_30/tile_m6dYq8wGj5U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_878QAjksX9k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_5AGXayBwZCM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_95lAfaJ_1o4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_RnIje33rRLY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_iFHrjLE08Vg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_xYnut80Y-zE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_HdPOmocD19w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_UlfQI_P7vco.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_G2nePRpiK6Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_jVMuoz3FmBA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile__lHwd88WlAk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_uA6JC01J-II.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_n6WL2zzxprU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_e7OyCSqpV-c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_MeDOXFSvFNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_uptv8iGJBi4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_PiL6Joun_-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_j1y0l7fP-Sk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_NYfDvlcPv0c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_KhF1V8P1Z20.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_wcCkxNyBXr4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2020_06_30/tile_MJ1U3NpVqR8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2020_06_30/tile_8AHkyfOa04w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2020_06_30/tile_25mk9l1sSUU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2020_06_30/tile_ezNXcekn93E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2020_06_30/tile_xV-T0Fy3KFU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2020_06_30/tile_zcYd-KLAVFM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa34_2020_06_30/tile_OAmzV52YF2w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa34_2020_06_30/tile_20SedX5wtg8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2020_06_30/tile_6RNE_2tn9Z0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2020_06_30/tile_cI_89vaQokA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2020_06_30/tile_5sFxLBA_Iqk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2020_06_30/tile_SrA3hNKGZ64.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2020_06_30/tile_Dd4GpvS8Ge0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2020_06_30/tile_YNgafwyhidk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2020_06_30/tile_t8LbcNrkWrE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2020_06_30/tile_SCFPnVfUql8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2020_06_30/tile_xVbV88yWUhE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2020_06_30/tile_I1OMvjAHoc8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2020_06_30/tile_MVyILXNgnas.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2020_06_30/tile_dPY0-2oQmtU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2020_06_30/tile_LBRA9z5hNF4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2020_06_30/tile_XacG0jFDyH8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2020_06_30/tile_5WXILtuOQbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2020_06_30/tile_uedmM83ZqRs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2020_06_30/tile_H42ZeH4gFtI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2020_06_30/tile_4PJBpAUrAqw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2020_06_30/tile_y4YR_QsaTcQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2020_06_30/tile_72OrTmz8LwY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2020_06_30/tile_45CwXin5MhE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2020_06_30/tile_rxgV4aC8GWg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2020_06_30/tile_hn2htg6Y1KQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2020_06_30/tile_R6kRjlEAS2Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2020_06_30/tile_PZ7IGn7F_pE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa7c_2020_06_30/tile_8Eo7jYPCsEE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_bvvyF6CZ3v0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_YJmq-27yWK4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_N77B5UsvXQI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_0qjzxzBybu8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_Cfv04geebOc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_wea4hDdk5sM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_oaUjV8M2qVY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_yzO245pSh50.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_YxKi3PRdpE8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_98ZSThIgVNw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_UbaNaWeQQhY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_uys4W-tzfws.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_-6WIgLec_Aw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_uZXb1B-6-Pc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_zaqnXBBelc4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_r9EbIgjkq3Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_pdkAbSnbGXM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2020_06_30/tile_V4S8RzrAoRM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_ncdQ401nuQM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_tnaJo4bzUSI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_3PP94IvU6LU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_OGCSC_GRupA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_xxz7CbfxxI0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_ovl1QE5MZxM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_-v3t1LPEWFE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_2UU7iRXs0pw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_Kaej5n7IJdI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_iM8BvsuXYgo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_bdveB5gKAR8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_Rr4IwM4hAR4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_dr5N5SqIUdc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_4WsINMFjQyA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_OuCHhfHiIiM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_m9WKjXykcrw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_za6lbJmtBNA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_zmVokDgk-Qc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_BvnC-jJY7wY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_n8T27F4DPvU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_t3tWUMfnM1s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile___2Ly15PD-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_N9fYMYCIDXM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_8u4vsdT3ezc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_zJgy7xJ6s14.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_XwIpcN0I42o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_KYwaBtt8bmc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_CgaoNS4tIPY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_BhG1_EGrxp0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_Dmbab34e-DM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_LoB42QzAKR4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_k_rYgCSlHoc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_Sm-4CiwChiU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_YdOJsuQ3NTE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_Wr5S4gpj6PI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_2sNXli1EiBM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_X4FrkiIFSFw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_oU9nU8lnzeU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_bu142fanRU4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_320NPFa_c6I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_ap6MmlOc2yU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_RnzVPV79IwM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_CjC2YokAYNM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_fvLCqLHC2gg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_fRnyS-6OCAI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_OWU6LzCOOSA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile__OvnbPSQ9ns.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_UXH8WmphVWQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2021_06_30/tile_CXR3m-7oxhw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_ldBb0Tr1ZwM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_rFvCO4Ah9no.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_5e0MejX9ZxU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_DBuHBh-X59s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_jgxruYmBwN8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_r8fBOaQdJ8E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_qGc0-6zuNLc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_Vdad6TSu3pw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_VRnX6WYY5aY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_FNTgkpAGzbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_2lmC2Q0_NGI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_31fe1gaUUJs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_oBoJG9S-IuU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_Qs4Ftn6U57k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_DzrfLr9c23M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_vQXO29VvamQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_YcYIvV8wU2o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_-lw3NKgt5CE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_eZ0UI9WkUa4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_GE7bCP4ZofQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_Dv6dcOWnI2s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_oWuYRxYPWts.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_9czOQj56LgE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_1jb0MV6ft44.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_GvjN6ss4IjM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_l2Ijk61lArY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_9_oKvfHROyI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_8prFLjh2fCA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_dvw7RgB7zV8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_oA1IXzGzBwU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_NmTWC35JRWA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2021_06_30/tile_EMjG3YF7g-U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2021_06_30/tile_VKlB1vsTo1k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2021_06_30/tile_iLYSjIvoV7c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2021_06_30/tile_g5NW4OyrF_w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2021_06_30/tile_UhcGGXD9HTA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2021_06_30/tile_IaKubeBxk-o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2021_06_30/tile_xlHKnJlfjjI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2021_06_30/tile_c5GJn9lqQlg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2021_06_30/tile_A1fOd0d6vfw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2021_06_30/tile_p8fmD97oUw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2021_06_30/tile_j-0-feR5x-Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2021_06_30/tile_D0Ykpy1Q3dE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2021_06_30/tile_fb49UA89wsE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2021_06_30/tile_VxTjtzLnz5s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2021_06_30/tile_MjZ7ijC2MMQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2021_06_30/tile_tI_DZyIguZw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2021_06_30/tile_2CBafV7FlMU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9ec_2021_06_30/tile_o55i51eydaQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_0A2gJ29ZmZM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_ZhnhFasyWJQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_Af0yq0hni6I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_Fy_29KcjBV4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_DyawyUvpKNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_gUpSW8_n9tk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_WMfRllKRxBs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_lfAYMhyogzA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_b3D3Gy-xk54.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_414GXSY0u0k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_AfMu6nphNKM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_xxXoMLneFKQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_lO3dSzyBHVc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_da5aF9PdeiY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_7Zvh8BTRDDM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_Ei6ZwffbYpg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_BGo4763ZAe4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_-mIbXBC7-fM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_C55GT80DLy4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_zZUrRAwJHcs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_rfD3ZaPrkvA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_LLfSbGafW2s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_6GEy0NhrfLs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2021_06_30/tile_oC7JciLmHEI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_-PV0lk8Wcy0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_BbdgT1OZ4xA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_OebMUMSKp58.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_xAHpfDS_tcY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_Zz62K_GjAEg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_9P8rv_pKqQo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_CiL8thFOPLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_SWbvqMiMruQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_qOmIXpDy4tI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_GalsFR1ogcE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_Xd4BfprZwWo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_KQNKVCrMrew.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_ziyY0HB_UIk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_jOY4cOkLlOI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_Zo-kK8qGZ_A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_At-ZNnlNAdc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2021_06_30/tile_bo1ZdxoCCwQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_F7op2ob98HQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_YuT9ce3zbXY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_dy3csCGSuNw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_BgGCZSUVKmE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_S9Z4shzmIsE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_vDGSmkdV7os.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_LpZkF1XEY3o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_vI4LhyMPWeg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_YPV1DLqSPIM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_notmacDKViI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_SyLdYKgXsIo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_x7Kq-k7XyAE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_3Q2vR4iJkm8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_sfCXwt6jEz8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_mSVN8yZI-Uk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_h_aPHCxQxSA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_iAKMUFyZAgE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_fSa4zxMdvmc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_szHgiDfUXTY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_66zEbCqh7LA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_HppyIg2uSME.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_Ry0ghFfa2jo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_Dvi1vUuf3Es.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_uahk6r8D0Wo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_2XHdvyceIE0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_oDQaqWtZuxs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_sdYsbVcgUh8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_yqFpuXzmrnw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_n8iP_OQUSLw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_gXWjVVfGCfk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_EV4_heFBtbs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_iNLBAvgkbTg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_P_yj4kitRUw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_eVz79CwsG7U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_oTzNnOPQcss.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_FJlshPGyBoo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_bvz58Sj6DgU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_xGCy64-7DpE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_d-Gf9Hjsfq8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_PDeI1LBjxz8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_gyMYOk8CKtY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_9YDjRVGgNDU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_W-lHKI0gAro.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_iWvpRfo_44w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_lhMLAAYgmBE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_5KlVTvp3Ud8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_26937ENrox4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_4NozDtFdows.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2021_06_30/tile_pgF_G4DWa8E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2021_06_30/tile_6l8nQqpP3Fs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2021_06_30/tile_Fvzd5OeOzH8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2021_06_30/tile_oJJJQx20xus.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2021_06_30/tile_ndb8nuuMFQE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2021_06_30/tile_xyE-tv1vbpg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2021_06_30/tile_fJmNd-vIC8I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2021_06_30/tile_uyjeDqZaf00.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2021_06_30/tile_ohY41G3cspQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2021_06_30/tile_8bxtBmYoeig.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2021_06_30/tile_r3cl8mRaJw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2021_06_30/tile_aOIVRthWYlM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2021_06_30/tile_ca5H2wWNgOk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2021_06_30/tile_7IbjBqHuIng.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2021_06_30/tile_sYxHTLvk0ZQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_edtHlMNdHlk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_5bJrF7sLxoY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_X2U3Wb9Yt2w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_ZMuqD-gMYds.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_M5R2IAMwhZw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_3At_utvBFVU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_yD-5IqzlIt4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_zlZZ6C20Xiw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_hrPqtg2pzLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_z5EXKgocvxo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_4EjvsUke-gg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_ZiMtkI2Qi3c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_hk2xZot2PXE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_0WuKT_oH-c4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_zUe6xvSLPZ8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_ra5oBq3ZhqM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_tAS97RqKbsY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_Qmj1dsXHujU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_ThrufapKc8c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_-S4mAuWhX0s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_a2Ceu-awLLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_Qf2T9y2X8NQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_bQpozx7M-lU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_cek61MplDpc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_fTXx7GhMV_M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_cFkjr0PDIQg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_fnUcJei0LlA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_F43y26WmrH4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_z4UdTx58XLg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_SYDTGycWzaI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_hN6DkXzYW2M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_P0LL9oS_HA8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_YBzNNWaiQTw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_gOzae0pJwvs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2021_06_30/tile_4o-b8zJb58o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa34_2021_06_30/tile_OAmzV52YF2w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa34_2021_06_30/tile_20SedX5wtg8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2021_06_30/tile_I1OMvjAHoc8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2021_06_30/tile_MVyILXNgnas.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2021_06_30/tile_dPY0-2oQmtU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2021_06_30/tile_LBRA9z5hNF4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2021_06_30/tile_XacG0jFDyH8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2021_06_30/tile_5WXILtuOQbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2021_06_30/tile_uedmM83ZqRs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2021_06_30/tile_H42ZeH4gFtI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2021_06_30/tile_4PJBpAUrAqw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2021_06_30/tile_y4YR_QsaTcQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2021_06_30/tile_72OrTmz8LwY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2021_06_30/tile_45CwXin5MhE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2021_06_30/tile_rxgV4aC8GWg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2021_06_30/tile_hn2htg6Y1KQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2021_06_30/tile_R6kRjlEAS2Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2021_06_30/tile_PZ7IGn7F_pE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_nPkNNFAZKLM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_9v8P-w1N6_g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_LCXbkT5Dbv4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_rO4GFLbHNqk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_SjJwsggAeY4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_HwzR7aY6WIo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_zikBG5UPINA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_-0lEnLgfrGU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_VyEdJOyGwNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_JyXtw1WXzu4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile__qWrETqMx48.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_hv51HgzPQjc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_DWzHgPsDu_c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_CRQSl5Iz7Rk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_TcomU2q_loQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_bSSp3yNUssU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_EtzjtiiE4ZY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_siUk0QP_z8g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_otJdZNEr8X8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_m_g0A3UNuGY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_rrzf2r3R5Js.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_os-FTpdD5sc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_lAXVJ1CZ8PE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_f6mhLCvACJk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_hGDdIkHOdLU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_MUTAkRKvdHI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_EdNson2UbQc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_GTMv-a6-QQI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_Me2UovXVGlw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_kVYBvRC38zs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_FUN5e6yclBM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_whdVajibyDI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_igkH8p44L3U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_obdfPw2Hu3s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_6ITL_yjNmww.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_TvXw-dGPgqA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_kjMdbwQjorw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_bfkzc-Hocc0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_xU55vI4txgU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_iELDzowUH7M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_XhrqU2iikHQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_BiuWipHcYKQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2021_06_30/tile_v0HPhTzrt20.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_878QAjksX9k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_5AGXayBwZCM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_95lAfaJ_1o4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_RnIje33rRLY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_iFHrjLE08Vg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_xYnut80Y-zE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_HdPOmocD19w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_UlfQI_P7vco.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_G2nePRpiK6Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_jVMuoz3FmBA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile__lHwd88WlAk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_uA6JC01J-II.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_n6WL2zzxprU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_e7OyCSqpV-c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_MeDOXFSvFNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_uptv8iGJBi4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_PiL6Joun_-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_j1y0l7fP-Sk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_NYfDvlcPv0c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_KhF1V8P1Z20.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_wcCkxNyBXr4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2021_06_30/tile_MJ1U3NpVqR8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa7c_2021_06_30/tile_8Eo7jYPCsEE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_bIYrX_YAxwk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_Hu5c9sSMr3A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_vrjZGrwBCA0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_unQMAvozNRk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_0xI9fVduyZE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_wzA4WQgAvrE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_Y8ImHQMAcYA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_58pnWEGSKt8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_yY2x2fammoM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_8nVOcGNu2IA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_RlsKT6Jf6wM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_pq8UH-HEOQg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_VqOmW5kz1W0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_PKm8xyhmIZc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_b_WBA1b7eZA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_FeeFcJAZbYc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_fWsQe-ZeEe8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_UPxsMf5HJWI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_SVIOaY507UM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_QM9EXlxbeBA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_a9oLR3z8Vm0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_R84NkdXV2Z0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_nNA6Pwk-M3U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_jxm7UNj7-Rc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_rc_yujmCW-U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_2OIs0gqeq1k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_Gex4QzSIKy0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_Hc-UpwT69ws.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_gVc0TpxZv7E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_MISuRqfSgeo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_fvrUa3e43x4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_I5fa0cA523A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_JQxlFUS4NfQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_vdnHmna7Ifo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_NmJNE6dWWos.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_xHWoK3gkHe8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_u8FFj7cZHIQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_QWcgi3AtUuU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_4kQAI2-RaC4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_ROOd-NnESME.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_R4y_itQ4hd4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_VFjB7PY-h3w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_MFmAF07BBrc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_F14sWKylN6c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_qUphd3XFpW0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2021_06_30/tile_QtvRPAbpGLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_PGmIEWWtvYg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_lrb8RyXXxUs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_crBc3qKvc_c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_pC6GZFISPXI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_jxnI9CDibDk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_HxMnqi_eHyc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_HwWmPLz2-Vc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_U1sP7RWC1gA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_oHUY41RqYrU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_8qViUzHeLCw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_MuJoJ-9MANU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_-Cuet1TxnNo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_FrFxz3X-7Mo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_AWio5XBXSAc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_9dX3Ngf4Z-4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_gzcQRuzEqLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_0J1CoSS6iPg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_scIfdKTURXA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_ap0j3H6RE1g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile__oLuYSkg12A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_zOGOLNWhXco.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_x73vJhmPBaI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_4Wt4RqUDYv8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_3DxVMYeRslw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_bJ2k97ym_Kw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_yIGOSSIzoMg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_Ig2W_gviuTI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_1YAFw6W52-E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_TF4q8zPmRRc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_mJYLHk7hDEc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_2BM8UBy62go.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_THLy2cMWTJE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_F68y1CtJLsg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_YevJTKbTzTw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_X7Vdb4oIr2Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_xaFmlC3lb-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_DoySuYmtk-w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_KYDEwg89lH0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_vy7ZbKCC8Lk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_W2vgIrSqXIA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_0C3XV_w3Hcc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_U8Xo8TRr30Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_H-lLEvnmoLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_7-krjk7IdVY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_Fsy2GZyiqT8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_Wy2x2VSavKU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_ZxrWWCThOi4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2021_06_30/tile_GSz_zarI5kI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2021_06_30/tile_rMqJeLTuHRE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2021_06_30/tile_nfD2LYI6nYo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2021_06_30/tile_Hjd3hvqGQ1s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2021_06_30/tile_Z37cKCmA-cM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2021_06_30/tile_cFdPOG9KpSE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2021_06_30/tile_GdNxjPqD76I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2021_06_30/tile_o4dsAI5yWBg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2021_06_30/tile_F6QKTYMZ3wI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2021_06_30/tile_5tWtjSZGz_k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2021_06_30/tile_gaHpMg_bTWU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_VPcv0zeEUUY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_Qgqg0VZsvoo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_l6o10K2govA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_2fJYus8UkwQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_-ElEld5rt5E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_iZkKS0q50w4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_gS5D6klKAkE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_EtmfDSzhAw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_hqTAUj8QqPw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_88jQlqXOVf8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_mgmMDaDtiUQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_nDRZTLCQqvI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_6ViRnrkFi_Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_4bKe6c_tbQo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_RjKOUXqeAsU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_cfa0maP-1FE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_mcPmzf75ToE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_gUx9RYKOj8w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_p_4EDz0m5iU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile__iS3cAV0l1Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_F66qhzaKdUg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_aIt-UR2UqDE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_0chzaXGxohY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_xDRoay6WAbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_T-EVYoBtsvk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_1_M0gDok2-0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_GcKlG8oRV6o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_6RRVAfi_A6Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_3SoNvBqku9E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_iDsfCsFq_II.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_MpAXOG-IkOs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_IdsknuIsqQE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_YcFMiBDiD60.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_6pQK9PWLHRw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_BEpPJrteTbg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_Zw2iwVxp9ok.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_jDOBX14ESqY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_YBUOyhJE9C0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_OC2WfdcNv7I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_atQiFq44XNY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_dFh6u3eGS14.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_EbYnhHZcsoM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_HEYB6rB4afs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_ufajWPtQrts.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_qrp1SrVUIic.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_sCoH8x5AbKI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_mCqEfn-4q4A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_PqI_YrPNSWE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2021_06_30/tile_m6dYq8wGj5U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2021_06_30/tile_RxMvIrdiSa4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2021_06_30/tile_eJzTsOJZsUg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2021_06_30/tile_cOL6HYvY8VA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_9z5qs7azJEg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_UeFS52yPtVE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_-UHrdUFyL5Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_jU67INtOoPE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_ibeuqkRRCDI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile__5iKyN8m6Og.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_dvywPun0dsA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_VWJc22h_hBQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_8_DRwoUsU9Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_NIVD-Z4zMU8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_w4XApkuShgs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_1yMS-7nDS5s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_lhR5sXnbzRk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_xhU82Q9ccg8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_GlSCj7IFwnY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_P_B1sisDlRY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_eXoQAjVcOYQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_vW2MJxoFgMk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_Sq3-rJUxsNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_xQ-44Zay7aA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_xDIcGmh6HVg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_CWZrLuhZaro.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_I67YZsLB-kw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_s9WQ6imuVJI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_Wdo7UBfds04.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_kYInGFjnjWA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_NGJz-evoHWQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_I_aWqMwULzI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2021_06_30/tile_IN36N6xwaq0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2021_06_30/tile_6RNE_2tn9Z0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2021_06_30/tile_cI_89vaQokA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2021_06_30/tile_5sFxLBA_Iqk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2021_06_30/tile_SrA3hNKGZ64.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2021_06_30/tile_Dd4GpvS8Ge0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2021_06_30/tile_YNgafwyhidk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2021_06_30/tile_t8LbcNrkWrE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2021_06_30/tile_SCFPnVfUql8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2021_06_30/tile_xVbV88yWUhE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_bvvyF6CZ3v0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_YJmq-27yWK4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_N77B5UsvXQI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_0qjzxzBybu8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_Cfv04geebOc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_wea4hDdk5sM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_oaUjV8M2qVY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_yzO245pSh50.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_YxKi3PRdpE8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_98ZSThIgVNw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_UbaNaWeQQhY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_uys4W-tzfws.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_-6WIgLec_Aw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_uZXb1B-6-Pc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_zaqnXBBelc4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_r9EbIgjkq3Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_pdkAbSnbGXM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2021_06_30/tile_V4S8RzrAoRM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2021_06_30/tile_8AHkyfOa04w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2021_06_30/tile_25mk9l1sSUU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2021_06_30/tile_ezNXcekn93E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2021_06_30/tile_xV-T0Fy3KFU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2021_06_30/tile_zcYd-KLAVFM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_-PV0lk8Wcy0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_BbdgT1OZ4xA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_OebMUMSKp58.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_xAHpfDS_tcY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_Zz62K_GjAEg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_9P8rv_pKqQo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_CiL8thFOPLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_SWbvqMiMruQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_qOmIXpDy4tI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_GalsFR1ogcE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_Xd4BfprZwWo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_KQNKVCrMrew.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_ziyY0HB_UIk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_jOY4cOkLlOI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_Zo-kK8qGZ_A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_At-ZNnlNAdc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2022_06_30/tile_bo1ZdxoCCwQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9ec_2022_06_30/tile_o55i51eydaQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_ncdQ401nuQM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_tnaJo4bzUSI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_3PP94IvU6LU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_OGCSC_GRupA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_xxz7CbfxxI0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_ovl1QE5MZxM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_-v3t1LPEWFE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_2UU7iRXs0pw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_Kaej5n7IJdI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_iM8BvsuXYgo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_bdveB5gKAR8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_Rr4IwM4hAR4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_dr5N5SqIUdc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_4WsINMFjQyA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_OuCHhfHiIiM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_m9WKjXykcrw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_za6lbJmtBNA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_zmVokDgk-Qc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_BvnC-jJY7wY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_n8T27F4DPvU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_t3tWUMfnM1s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile___2Ly15PD-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_N9fYMYCIDXM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_8u4vsdT3ezc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_zJgy7xJ6s14.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_XwIpcN0I42o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_KYwaBtt8bmc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_CgaoNS4tIPY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_BhG1_EGrxp0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_Dmbab34e-DM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_LoB42QzAKR4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_k_rYgCSlHoc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_Sm-4CiwChiU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_YdOJsuQ3NTE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_Wr5S4gpj6PI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_2sNXli1EiBM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_X4FrkiIFSFw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_oU9nU8lnzeU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_bu142fanRU4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_320NPFa_c6I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_ap6MmlOc2yU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_RnzVPV79IwM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_CjC2YokAYNM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_fvLCqLHC2gg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_fRnyS-6OCAI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_OWU6LzCOOSA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile__OvnbPSQ9ns.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_UXH8WmphVWQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2022_06_30/tile_CXR3m-7oxhw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f92c_2022_06_30/tile_uZL-huit1Wc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f92c_2022_06_30/tile_kx82idbxdS0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_F7op2ob98HQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_YuT9ce3zbXY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_dy3csCGSuNw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_BgGCZSUVKmE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_S9Z4shzmIsE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_vDGSmkdV7os.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_LpZkF1XEY3o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_vI4LhyMPWeg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_YPV1DLqSPIM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_notmacDKViI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_SyLdYKgXsIo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_x7Kq-k7XyAE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_3Q2vR4iJkm8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_sfCXwt6jEz8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_mSVN8yZI-Uk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_h_aPHCxQxSA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_iAKMUFyZAgE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_fSa4zxMdvmc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_szHgiDfUXTY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_66zEbCqh7LA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_HppyIg2uSME.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_Ry0ghFfa2jo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_Dvi1vUuf3Es.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_uahk6r8D0Wo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_2XHdvyceIE0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_oDQaqWtZuxs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_sdYsbVcgUh8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_yqFpuXzmrnw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_n8iP_OQUSLw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_gXWjVVfGCfk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_EV4_heFBtbs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_iNLBAvgkbTg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_P_yj4kitRUw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_eVz79CwsG7U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_oTzNnOPQcss.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_FJlshPGyBoo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_bvz58Sj6DgU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_xGCy64-7DpE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_d-Gf9Hjsfq8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_PDeI1LBjxz8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_gyMYOk8CKtY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_9YDjRVGgNDU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_W-lHKI0gAro.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_iWvpRfo_44w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_lhMLAAYgmBE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_5KlVTvp3Ud8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_26937ENrox4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_4NozDtFdows.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2022_06_30/tile_pgF_G4DWa8E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2022_06_30/tile_VKlB1vsTo1k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2022_06_30/tile_iLYSjIvoV7c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2022_06_30/tile_g5NW4OyrF_w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2022_06_30/tile_UhcGGXD9HTA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2022_06_30/tile_IaKubeBxk-o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2022_06_30/tile_xlHKnJlfjjI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2022_06_30/tile_c5GJn9lqQlg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2022_06_30/tile_A1fOd0d6vfw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2022_06_30/tile_p8fmD97oUw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2022_06_30/tile_j-0-feR5x-Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2022_06_30/tile_D0Ykpy1Q3dE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2022_06_30/tile_fb49UA89wsE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2022_06_30/tile_VxTjtzLnz5s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2022_06_30/tile_MjZ7ijC2MMQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2022_06_30/tile_tI_DZyIguZw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2022_06_30/tile_2CBafV7FlMU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_edtHlMNdHlk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_5bJrF7sLxoY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_X2U3Wb9Yt2w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_ZMuqD-gMYds.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_M5R2IAMwhZw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_3At_utvBFVU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_yD-5IqzlIt4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_zlZZ6C20Xiw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_hrPqtg2pzLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_z5EXKgocvxo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_4EjvsUke-gg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_ZiMtkI2Qi3c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_hk2xZot2PXE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_0WuKT_oH-c4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_zUe6xvSLPZ8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_ra5oBq3ZhqM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_tAS97RqKbsY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_Qmj1dsXHujU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_ThrufapKc8c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_-S4mAuWhX0s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_a2Ceu-awLLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_Qf2T9y2X8NQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_bQpozx7M-lU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_cek61MplDpc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_fTXx7GhMV_M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_cFkjr0PDIQg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_fnUcJei0LlA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_F43y26WmrH4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_z4UdTx58XLg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_SYDTGycWzaI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_hN6DkXzYW2M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_P0LL9oS_HA8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_YBzNNWaiQTw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_gOzae0pJwvs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2022_06_30/tile_4o-b8zJb58o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2022_06_30/tile_6l8nQqpP3Fs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2022_06_30/tile_Fvzd5OeOzH8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2022_06_30/tile_oJJJQx20xus.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2022_06_30/tile_ndb8nuuMFQE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2022_06_30/tile_xyE-tv1vbpg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2022_06_30/tile_fJmNd-vIC8I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2022_06_30/tile_uyjeDqZaf00.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2022_06_30/tile_ohY41G3cspQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2022_06_30/tile_8bxtBmYoeig.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2022_06_30/tile_r3cl8mRaJw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2022_06_30/tile_aOIVRthWYlM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2022_06_30/tile_ca5H2wWNgOk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2022_06_30/tile_7IbjBqHuIng.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2022_06_30/tile_sYxHTLvk0ZQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_0A2gJ29ZmZM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_ZhnhFasyWJQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_Af0yq0hni6I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_Fy_29KcjBV4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_DyawyUvpKNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_gUpSW8_n9tk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_WMfRllKRxBs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_lfAYMhyogzA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_b3D3Gy-xk54.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_414GXSY0u0k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_AfMu6nphNKM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_xxXoMLneFKQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_lO3dSzyBHVc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_da5aF9PdeiY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_7Zvh8BTRDDM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_Ei6ZwffbYpg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_BGo4763ZAe4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_-mIbXBC7-fM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_C55GT80DLy4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_zZUrRAwJHcs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_rfD3ZaPrkvA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_LLfSbGafW2s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_6GEy0NhrfLs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2022_06_30/tile_oC7JciLmHEI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_ldBb0Tr1ZwM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_rFvCO4Ah9no.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_5e0MejX9ZxU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_DBuHBh-X59s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_jgxruYmBwN8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_r8fBOaQdJ8E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_qGc0-6zuNLc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_Vdad6TSu3pw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_VRnX6WYY5aY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_FNTgkpAGzbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_2lmC2Q0_NGI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_31fe1gaUUJs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_oBoJG9S-IuU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_Qs4Ftn6U57k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_DzrfLr9c23M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_vQXO29VvamQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_YcYIvV8wU2o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_-lw3NKgt5CE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_eZ0UI9WkUa4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_GE7bCP4ZofQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_Dv6dcOWnI2s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_oWuYRxYPWts.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_9czOQj56LgE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_1jb0MV6ft44.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_GvjN6ss4IjM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_l2Ijk61lArY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_9_oKvfHROyI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_8prFLjh2fCA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_dvw7RgB7zV8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_oA1IXzGzBwU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_NmTWC35JRWA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2022_06_30/tile_EMjG3YF7g-U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2022_06_30/tile_6RNE_2tn9Z0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2022_06_30/tile_cI_89vaQokA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2022_06_30/tile_5sFxLBA_Iqk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2022_06_30/tile_SrA3hNKGZ64.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2022_06_30/tile_Dd4GpvS8Ge0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2022_06_30/tile_YNgafwyhidk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2022_06_30/tile_t8LbcNrkWrE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2022_06_30/tile_SCFPnVfUql8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2022_06_30/tile_xVbV88yWUhE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2022_06_30/tile_RxMvIrdiSa4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2022_06_30/tile_eJzTsOJZsUg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2022_06_30/tile_cOL6HYvY8VA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_nPkNNFAZKLM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_9v8P-w1N6_g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_LCXbkT5Dbv4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_rO4GFLbHNqk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_SjJwsggAeY4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_HwzR7aY6WIo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_zikBG5UPINA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_-0lEnLgfrGU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_VyEdJOyGwNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_JyXtw1WXzu4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile__qWrETqMx48.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_hv51HgzPQjc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_DWzHgPsDu_c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_CRQSl5Iz7Rk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_TcomU2q_loQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_bSSp3yNUssU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_EtzjtiiE4ZY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_siUk0QP_z8g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_otJdZNEr8X8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_m_g0A3UNuGY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_rrzf2r3R5Js.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_os-FTpdD5sc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_lAXVJ1CZ8PE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_f6mhLCvACJk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_hGDdIkHOdLU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_MUTAkRKvdHI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_EdNson2UbQc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_GTMv-a6-QQI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_Me2UovXVGlw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_kVYBvRC38zs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_FUN5e6yclBM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_whdVajibyDI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_igkH8p44L3U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_obdfPw2Hu3s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_6ITL_yjNmww.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_TvXw-dGPgqA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_kjMdbwQjorw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_bfkzc-Hocc0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_xU55vI4txgU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_iELDzowUH7M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_XhrqU2iikHQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_BiuWipHcYKQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2022_06_30/tile_v0HPhTzrt20.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa7c_2022_06_30/tile_8Eo7jYPCsEE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_VPcv0zeEUUY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_Qgqg0VZsvoo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_l6o10K2govA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_2fJYus8UkwQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_-ElEld5rt5E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_iZkKS0q50w4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_gS5D6klKAkE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_EtmfDSzhAw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_hqTAUj8QqPw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_88jQlqXOVf8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_mgmMDaDtiUQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_nDRZTLCQqvI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_6ViRnrkFi_Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_4bKe6c_tbQo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_RjKOUXqeAsU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_cfa0maP-1FE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_mcPmzf75ToE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_gUx9RYKOj8w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_p_4EDz0m5iU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile__iS3cAV0l1Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_F66qhzaKdUg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_aIt-UR2UqDE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_0chzaXGxohY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_xDRoay6WAbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_T-EVYoBtsvk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_1_M0gDok2-0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_GcKlG8oRV6o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_6RRVAfi_A6Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_3SoNvBqku9E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_iDsfCsFq_II.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_MpAXOG-IkOs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_IdsknuIsqQE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_YcFMiBDiD60.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_6pQK9PWLHRw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_BEpPJrteTbg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_Zw2iwVxp9ok.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_jDOBX14ESqY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_YBUOyhJE9C0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_OC2WfdcNv7I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_atQiFq44XNY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_dFh6u3eGS14.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_EbYnhHZcsoM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_HEYB6rB4afs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_ufajWPtQrts.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_qrp1SrVUIic.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_sCoH8x5AbKI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_mCqEfn-4q4A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_PqI_YrPNSWE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2022_06_30/tile_m6dYq8wGj5U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2022_06_30/tile_rMqJeLTuHRE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2022_06_30/tile_nfD2LYI6nYo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2022_06_30/tile_Hjd3hvqGQ1s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2022_06_30/tile_Z37cKCmA-cM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2022_06_30/tile_cFdPOG9KpSE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2022_06_30/tile_GdNxjPqD76I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2022_06_30/tile_o4dsAI5yWBg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2022_06_30/tile_F6QKTYMZ3wI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2022_06_30/tile_5tWtjSZGz_k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2022_06_30/tile_gaHpMg_bTWU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_9z5qs7azJEg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_UeFS52yPtVE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_-UHrdUFyL5Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_jU67INtOoPE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_ibeuqkRRCDI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile__5iKyN8m6Og.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_dvywPun0dsA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_VWJc22h_hBQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_8_DRwoUsU9Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_NIVD-Z4zMU8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_w4XApkuShgs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_1yMS-7nDS5s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_lhR5sXnbzRk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_xhU82Q9ccg8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_GlSCj7IFwnY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_P_B1sisDlRY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_eXoQAjVcOYQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_vW2MJxoFgMk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_Sq3-rJUxsNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_xQ-44Zay7aA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_xDIcGmh6HVg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_CWZrLuhZaro.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_I67YZsLB-kw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_s9WQ6imuVJI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_Wdo7UBfds04.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_kYInGFjnjWA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_NGJz-evoHWQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_I_aWqMwULzI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2022_06_30/tile_IN36N6xwaq0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2022_06_30/tile_8AHkyfOa04w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2022_06_30/tile_25mk9l1sSUU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2022_06_30/tile_ezNXcekn93E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2022_06_30/tile_xV-T0Fy3KFU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2022_06_30/tile_zcYd-KLAVFM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2022_06_30/tile_I1OMvjAHoc8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2022_06_30/tile_MVyILXNgnas.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2022_06_30/tile_dPY0-2oQmtU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2022_06_30/tile_LBRA9z5hNF4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2022_06_30/tile_XacG0jFDyH8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2022_06_30/tile_5WXILtuOQbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2022_06_30/tile_uedmM83ZqRs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2022_06_30/tile_H42ZeH4gFtI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2022_06_30/tile_4PJBpAUrAqw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2022_06_30/tile_y4YR_QsaTcQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2022_06_30/tile_72OrTmz8LwY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2022_06_30/tile_45CwXin5MhE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2022_06_30/tile_rxgV4aC8GWg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2022_06_30/tile_hn2htg6Y1KQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2022_06_30/tile_R6kRjlEAS2Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2022_06_30/tile_PZ7IGn7F_pE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_bIYrX_YAxwk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_Hu5c9sSMr3A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_vrjZGrwBCA0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_unQMAvozNRk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_0xI9fVduyZE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_wzA4WQgAvrE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_Y8ImHQMAcYA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_58pnWEGSKt8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_yY2x2fammoM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_8nVOcGNu2IA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_RlsKT6Jf6wM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_pq8UH-HEOQg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_VqOmW5kz1W0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_PKm8xyhmIZc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_b_WBA1b7eZA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_FeeFcJAZbYc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_fWsQe-ZeEe8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_UPxsMf5HJWI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_SVIOaY507UM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_QM9EXlxbeBA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_a9oLR3z8Vm0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_R84NkdXV2Z0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_nNA6Pwk-M3U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_jxm7UNj7-Rc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_rc_yujmCW-U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_2OIs0gqeq1k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_Gex4QzSIKy0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_Hc-UpwT69ws.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_gVc0TpxZv7E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_MISuRqfSgeo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_fvrUa3e43x4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_I5fa0cA523A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_JQxlFUS4NfQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_vdnHmna7Ifo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_NmJNE6dWWos.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_xHWoK3gkHe8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_u8FFj7cZHIQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_QWcgi3AtUuU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_4kQAI2-RaC4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_ROOd-NnESME.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_R4y_itQ4hd4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_VFjB7PY-h3w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_MFmAF07BBrc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_F14sWKylN6c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_qUphd3XFpW0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2022_06_30/tile_QtvRPAbpGLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_878QAjksX9k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_5AGXayBwZCM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_95lAfaJ_1o4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_RnIje33rRLY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_iFHrjLE08Vg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_xYnut80Y-zE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_HdPOmocD19w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_UlfQI_P7vco.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_G2nePRpiK6Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_jVMuoz3FmBA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile__lHwd88WlAk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_uA6JC01J-II.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_n6WL2zzxprU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_e7OyCSqpV-c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_MeDOXFSvFNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_uptv8iGJBi4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_PiL6Joun_-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_j1y0l7fP-Sk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_NYfDvlcPv0c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_KhF1V8P1Z20.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_wcCkxNyBXr4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2022_06_30/tile_MJ1U3NpVqR8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa34_2022_06_30/tile_OAmzV52YF2w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa34_2022_06_30/tile_20SedX5wtg8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_bvvyF6CZ3v0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_YJmq-27yWK4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_N77B5UsvXQI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_0qjzxzBybu8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_Cfv04geebOc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_wea4hDdk5sM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_oaUjV8M2qVY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_yzO245pSh50.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_YxKi3PRdpE8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_98ZSThIgVNw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_UbaNaWeQQhY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_uys4W-tzfws.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_-6WIgLec_Aw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_uZXb1B-6-Pc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_zaqnXBBelc4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_r9EbIgjkq3Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_pdkAbSnbGXM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2022_06_30/tile_V4S8RzrAoRM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_PGmIEWWtvYg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_lrb8RyXXxUs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_crBc3qKvc_c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_pC6GZFISPXI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_jxnI9CDibDk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_HxMnqi_eHyc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_HwWmPLz2-Vc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_U1sP7RWC1gA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_oHUY41RqYrU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_8qViUzHeLCw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_MuJoJ-9MANU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_-Cuet1TxnNo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_FrFxz3X-7Mo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_AWio5XBXSAc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_9dX3Ngf4Z-4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_gzcQRuzEqLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_0J1CoSS6iPg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_scIfdKTURXA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_ap0j3H6RE1g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile__oLuYSkg12A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_zOGOLNWhXco.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_x73vJhmPBaI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_4Wt4RqUDYv8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_3DxVMYeRslw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_bJ2k97ym_Kw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_yIGOSSIzoMg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_Ig2W_gviuTI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_1YAFw6W52-E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_TF4q8zPmRRc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_mJYLHk7hDEc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_2BM8UBy62go.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_THLy2cMWTJE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_F68y1CtJLsg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_YevJTKbTzTw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_X7Vdb4oIr2Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_xaFmlC3lb-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_DoySuYmtk-w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_KYDEwg89lH0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_vy7ZbKCC8Lk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_W2vgIrSqXIA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_0C3XV_w3Hcc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_U8Xo8TRr30Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_H-lLEvnmoLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_7-krjk7IdVY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_Fsy2GZyiqT8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_Wy2x2VSavKU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_ZxrWWCThOi4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2022_06_30/tile_GSz_zarI5kI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2023_06_30/tile_VKlB1vsTo1k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2023_06_30/tile_iLYSjIvoV7c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2023_06_30/tile_g5NW4OyrF_w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2023_06_30/tile_UhcGGXD9HTA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2023_06_30/tile_IaKubeBxk-o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2023_06_30/tile_xlHKnJlfjjI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2023_06_30/tile_c5GJn9lqQlg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2023_06_30/tile_A1fOd0d6vfw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2023_06_30/tile_p8fmD97oUw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2023_06_30/tile_j-0-feR5x-Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2023_06_30/tile_D0Ykpy1Q3dE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2023_06_30/tile_fb49UA89wsE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2023_06_30/tile_VxTjtzLnz5s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2023_06_30/tile_MjZ7ijC2MMQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2023_06_30/tile_tI_DZyIguZw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9f4_2023_06_30/tile_2CBafV7FlMU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_0A2gJ29ZmZM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_ZhnhFasyWJQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_Af0yq0hni6I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_Fy_29KcjBV4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_DyawyUvpKNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_gUpSW8_n9tk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_WMfRllKRxBs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_lfAYMhyogzA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_b3D3Gy-xk54.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_414GXSY0u0k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_AfMu6nphNKM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_xxXoMLneFKQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_lO3dSzyBHVc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_da5aF9PdeiY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_7Zvh8BTRDDM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_Ei6ZwffbYpg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_BGo4763ZAe4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_-mIbXBC7-fM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_C55GT80DLy4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_zZUrRAwJHcs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_rfD3ZaPrkvA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_LLfSbGafW2s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_6GEy0NhrfLs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa1c_2023_06_30/tile_oC7JciLmHEI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_ncdQ401nuQM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_tnaJo4bzUSI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_3PP94IvU6LU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_OGCSC_GRupA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_xxz7CbfxxI0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_ovl1QE5MZxM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_-v3t1LPEWFE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_2UU7iRXs0pw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_Kaej5n7IJdI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_iM8BvsuXYgo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_bdveB5gKAR8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_Rr4IwM4hAR4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_dr5N5SqIUdc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_4WsINMFjQyA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_OuCHhfHiIiM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_m9WKjXykcrw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_za6lbJmtBNA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_zmVokDgk-Qc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_BvnC-jJY7wY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_n8T27F4DPvU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_t3tWUMfnM1s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile___2Ly15PD-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_N9fYMYCIDXM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_8u4vsdT3ezc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_zJgy7xJ6s14.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_XwIpcN0I42o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_KYwaBtt8bmc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_CgaoNS4tIPY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_BhG1_EGrxp0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_Dmbab34e-DM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_LoB42QzAKR4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_k_rYgCSlHoc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_Sm-4CiwChiU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_YdOJsuQ3NTE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_Wr5S4gpj6PI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_2sNXli1EiBM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_X4FrkiIFSFw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_oU9nU8lnzeU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_bu142fanRU4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_320NPFa_c6I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_ap6MmlOc2yU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_RnzVPV79IwM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_CjC2YokAYNM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_fvLCqLHC2gg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_fRnyS-6OCAI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_OWU6LzCOOSA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile__OvnbPSQ9ns.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_UXH8WmphVWQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa04_2023_06_30/tile_CXR3m-7oxhw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_F7op2ob98HQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_YuT9ce3zbXY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_dy3csCGSuNw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_BgGCZSUVKmE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_S9Z4shzmIsE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_vDGSmkdV7os.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_LpZkF1XEY3o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_vI4LhyMPWeg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_YPV1DLqSPIM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_notmacDKViI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_SyLdYKgXsIo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_x7Kq-k7XyAE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_3Q2vR4iJkm8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_sfCXwt6jEz8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_mSVN8yZI-Uk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_h_aPHCxQxSA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_iAKMUFyZAgE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_fSa4zxMdvmc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_szHgiDfUXTY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_66zEbCqh7LA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_HppyIg2uSME.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_Ry0ghFfa2jo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_Dvi1vUuf3Es.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_uahk6r8D0Wo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_2XHdvyceIE0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_oDQaqWtZuxs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_sdYsbVcgUh8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_yqFpuXzmrnw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_n8iP_OQUSLw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_gXWjVVfGCfk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_EV4_heFBtbs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_iNLBAvgkbTg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_P_yj4kitRUw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_eVz79CwsG7U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_oTzNnOPQcss.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_FJlshPGyBoo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_bvz58Sj6DgU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_xGCy64-7DpE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_d-Gf9Hjsfq8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_PDeI1LBjxz8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_gyMYOk8CKtY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_9YDjRVGgNDU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_W-lHKI0gAro.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_iWvpRfo_44w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_lhMLAAYgmBE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_5KlVTvp3Ud8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_26937ENrox4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_4NozDtFdows.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9fc_2023_06_30/tile_pgF_G4DWa8E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2023_06_30/tile_6l8nQqpP3Fs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2023_06_30/tile_Fvzd5OeOzH8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2023_06_30/tile_oJJJQx20xus.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2023_06_30/tile_ndb8nuuMFQE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2023_06_30/tile_xyE-tv1vbpg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2023_06_30/tile_fJmNd-vIC8I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2023_06_30/tile_uyjeDqZaf00.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2023_06_30/tile_ohY41G3cspQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2023_06_30/tile_8bxtBmYoeig.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2023_06_30/tile_r3cl8mRaJw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2023_06_30/tile_aOIVRthWYlM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2023_06_30/tile_ca5H2wWNgOk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2023_06_30/tile_7IbjBqHuIng.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f75c_2023_06_30/tile_sYxHTLvk0ZQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_ldBb0Tr1ZwM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_rFvCO4Ah9no.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_5e0MejX9ZxU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_DBuHBh-X59s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_jgxruYmBwN8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_r8fBOaQdJ8E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_qGc0-6zuNLc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_Vdad6TSu3pw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_VRnX6WYY5aY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_FNTgkpAGzbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_2lmC2Q0_NGI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_31fe1gaUUJs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_oBoJG9S-IuU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_Qs4Ftn6U57k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_DzrfLr9c23M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_vQXO29VvamQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_YcYIvV8wU2o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_-lw3NKgt5CE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_eZ0UI9WkUa4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_GE7bCP4ZofQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_Dv6dcOWnI2s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_oWuYRxYPWts.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_9czOQj56LgE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_1jb0MV6ft44.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_GvjN6ss4IjM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_l2Ijk61lArY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_9_oKvfHROyI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_8prFLjh2fCA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_dvw7RgB7zV8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_oA1IXzGzBwU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_NmTWC35JRWA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0ac_2023_06_30/tile_EMjG3YF7g-U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_edtHlMNdHlk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_5bJrF7sLxoY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_X2U3Wb9Yt2w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_ZMuqD-gMYds.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_M5R2IAMwhZw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_3At_utvBFVU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_yD-5IqzlIt4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_zlZZ6C20Xiw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_hrPqtg2pzLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_z5EXKgocvxo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_4EjvsUke-gg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_ZiMtkI2Qi3c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_hk2xZot2PXE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_0WuKT_oH-c4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_zUe6xvSLPZ8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_ra5oBq3ZhqM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_tAS97RqKbsY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_Qmj1dsXHujU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_ThrufapKc8c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_-S4mAuWhX0s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_a2Ceu-awLLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_Qf2T9y2X8NQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_bQpozx7M-lU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_cek61MplDpc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_fTXx7GhMV_M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_cFkjr0PDIQg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_fnUcJei0LlA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_F43y26WmrH4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_z4UdTx58XLg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_SYDTGycWzaI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_hN6DkXzYW2M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_P0LL9oS_HA8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_YBzNNWaiQTw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_gOzae0pJwvs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f754_2023_06_30/tile_4o-b8zJb58o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9ec_2023_06_30/tile_o55i51eydaQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f92c_2023_06_30/tile_uZL-huit1Wc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f92c_2023_06_30/tile_kx82idbxdS0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_-PV0lk8Wcy0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_BbdgT1OZ4xA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_OebMUMSKp58.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_xAHpfDS_tcY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_Zz62K_GjAEg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_9P8rv_pKqQo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_CiL8thFOPLI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_SWbvqMiMruQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_qOmIXpDy4tI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_GalsFR1ogcE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_Xd4BfprZwWo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_KQNKVCrMrew.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_ziyY0HB_UIk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_jOY4cOkLlOI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_Zo-kK8qGZ_A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_At-ZNnlNAdc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f9e4_2023_06_30/tile_bo1ZdxoCCwQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_nPkNNFAZKLM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_9v8P-w1N6_g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_LCXbkT5Dbv4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_rO4GFLbHNqk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_SjJwsggAeY4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_HwzR7aY6WIo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_zikBG5UPINA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_-0lEnLgfrGU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_VyEdJOyGwNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_JyXtw1WXzu4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile__qWrETqMx48.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_hv51HgzPQjc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_DWzHgPsDu_c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_CRQSl5Iz7Rk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_TcomU2q_loQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_bSSp3yNUssU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_EtzjtiiE4ZY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_siUk0QP_z8g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_otJdZNEr8X8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_m_g0A3UNuGY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_rrzf2r3R5Js.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_os-FTpdD5sc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_lAXVJ1CZ8PE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_f6mhLCvACJk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_hGDdIkHOdLU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_MUTAkRKvdHI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_EdNson2UbQc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_GTMv-a6-QQI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_Me2UovXVGlw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_kVYBvRC38zs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_FUN5e6yclBM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_whdVajibyDI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_igkH8p44L3U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_obdfPw2Hu3s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_6ITL_yjNmww.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_TvXw-dGPgqA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_kjMdbwQjorw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_bfkzc-Hocc0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_xU55vI4txgU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_iELDzowUH7M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_XhrqU2iikHQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_BiuWipHcYKQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa14_2023_06_30/tile_v0HPhTzrt20.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2023_06_30/tile_RxMvIrdiSa4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2023_06_30/tile_eJzTsOJZsUg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa54_2023_06_30/tile_cOL6HYvY8VA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2023_06_30/tile_rMqJeLTuHRE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2023_06_30/tile_nfD2LYI6nYo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2023_06_30/tile_Hjd3hvqGQ1s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2023_06_30/tile_Z37cKCmA-cM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2023_06_30/tile_cFdPOG9KpSE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2023_06_30/tile_GdNxjPqD76I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2023_06_30/tile_o4dsAI5yWBg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2023_06_30/tile_F6QKTYMZ3wI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2023_06_30/tile_5tWtjSZGz_k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa64_2023_06_30/tile_gaHpMg_bTWU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa34_2023_06_30/tile_OAmzV52YF2w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa34_2023_06_30/tile_20SedX5wtg8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_VPcv0zeEUUY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_Qgqg0VZsvoo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_l6o10K2govA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_2fJYus8UkwQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_-ElEld5rt5E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_iZkKS0q50w4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_gS5D6klKAkE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_EtmfDSzhAw4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_hqTAUj8QqPw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_88jQlqXOVf8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_mgmMDaDtiUQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_nDRZTLCQqvI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_6ViRnrkFi_Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_4bKe6c_tbQo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_RjKOUXqeAsU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_cfa0maP-1FE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_mcPmzf75ToE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_gUx9RYKOj8w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_p_4EDz0m5iU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile__iS3cAV0l1Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_F66qhzaKdUg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_aIt-UR2UqDE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_0chzaXGxohY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_xDRoay6WAbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_T-EVYoBtsvk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_1_M0gDok2-0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_GcKlG8oRV6o.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_6RRVAfi_A6Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_3SoNvBqku9E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_iDsfCsFq_II.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_MpAXOG-IkOs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_IdsknuIsqQE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_YcFMiBDiD60.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_6pQK9PWLHRw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_BEpPJrteTbg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_Zw2iwVxp9ok.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_jDOBX14ESqY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_YBUOyhJE9C0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_OC2WfdcNv7I.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_atQiFq44XNY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_dFh6u3eGS14.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_EbYnhHZcsoM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_HEYB6rB4afs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_ufajWPtQrts.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_qrp1SrVUIic.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_sCoH8x5AbKI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_mCqEfn-4q4A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_PqI_YrPNSWE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa0c_2023_06_30/tile_m6dYq8wGj5U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_PGmIEWWtvYg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_lrb8RyXXxUs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_crBc3qKvc_c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_pC6GZFISPXI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_jxnI9CDibDk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_HxMnqi_eHyc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_HwWmPLz2-Vc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_U1sP7RWC1gA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_oHUY41RqYrU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_8qViUzHeLCw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_MuJoJ-9MANU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_-Cuet1TxnNo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_FrFxz3X-7Mo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_AWio5XBXSAc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_9dX3Ngf4Z-4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_gzcQRuzEqLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_0J1CoSS6iPg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_scIfdKTURXA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_ap0j3H6RE1g.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile__oLuYSkg12A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_zOGOLNWhXco.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_x73vJhmPBaI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_4Wt4RqUDYv8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_3DxVMYeRslw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_bJ2k97ym_Kw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_yIGOSSIzoMg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_Ig2W_gviuTI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_1YAFw6W52-E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_TF4q8zPmRRc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_mJYLHk7hDEc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_2BM8UBy62go.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_THLy2cMWTJE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_F68y1CtJLsg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_YevJTKbTzTw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_X7Vdb4oIr2Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_xaFmlC3lb-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_DoySuYmtk-w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_KYDEwg89lH0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_vy7ZbKCC8Lk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_W2vgIrSqXIA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_0C3XV_w3Hcc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_U8Xo8TRr30Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_H-lLEvnmoLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_7-krjk7IdVY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_Fsy2GZyiqT8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_Wy2x2VSavKU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_ZxrWWCThOi4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa44_2023_06_30/tile_GSz_zarI5kI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_9z5qs7azJEg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_UeFS52yPtVE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_-UHrdUFyL5Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_jU67INtOoPE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_ibeuqkRRCDI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile__5iKyN8m6Og.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_dvywPun0dsA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_VWJc22h_hBQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_8_DRwoUsU9Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_NIVD-Z4zMU8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_w4XApkuShgs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_1yMS-7nDS5s.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_lhR5sXnbzRk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_xhU82Q9ccg8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_GlSCj7IFwnY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_P_B1sisDlRY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_eXoQAjVcOYQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_vW2MJxoFgMk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_Sq3-rJUxsNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_xQ-44Zay7aA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_xDIcGmh6HVg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_CWZrLuhZaro.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_I67YZsLB-kw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_s9WQ6imuVJI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_Wdo7UBfds04.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_kYInGFjnjWA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_NGJz-evoHWQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_I_aWqMwULzI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa74_2023_06_30/tile_IN36N6xwaq0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa7c_2023_06_30/tile_8Eo7jYPCsEE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_bvvyF6CZ3v0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_YJmq-27yWK4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_N77B5UsvXQI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_0qjzxzBybu8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_Cfv04geebOc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_wea4hDdk5sM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_oaUjV8M2qVY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_yzO245pSh50.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_YxKi3PRdpE8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_98ZSThIgVNw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_UbaNaWeQQhY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_uys4W-tzfws.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_-6WIgLec_Aw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_uZXb1B-6-Pc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_zaqnXBBelc4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_r9EbIgjkq3Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_pdkAbSnbGXM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa3c_2023_06_30/tile_V4S8RzrAoRM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2023_06_30/tile_8AHkyfOa04w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2023_06_30/tile_25mk9l1sSUU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2023_06_30/tile_ezNXcekn93E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2023_06_30/tile_xV-T0Fy3KFU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f09c_2023_06_30/tile_zcYd-KLAVFM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_bIYrX_YAxwk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_Hu5c9sSMr3A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_vrjZGrwBCA0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_unQMAvozNRk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_0xI9fVduyZE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_wzA4WQgAvrE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_Y8ImHQMAcYA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_58pnWEGSKt8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_yY2x2fammoM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_8nVOcGNu2IA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_RlsKT6Jf6wM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_pq8UH-HEOQg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_VqOmW5kz1W0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_PKm8xyhmIZc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_b_WBA1b7eZA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_FeeFcJAZbYc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_fWsQe-ZeEe8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_UPxsMf5HJWI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_SVIOaY507UM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_QM9EXlxbeBA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_a9oLR3z8Vm0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_R84NkdXV2Z0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_nNA6Pwk-M3U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_jxm7UNj7-Rc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_rc_yujmCW-U.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_2OIs0gqeq1k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_Gex4QzSIKy0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_Hc-UpwT69ws.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_gVc0TpxZv7E.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_MISuRqfSgeo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_fvrUa3e43x4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_I5fa0cA523A.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_JQxlFUS4NfQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_vdnHmna7Ifo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_NmJNE6dWWos.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_xHWoK3gkHe8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_u8FFj7cZHIQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_QWcgi3AtUuU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_4kQAI2-RaC4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_ROOd-NnESME.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_R4y_itQ4hd4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_VFjB7PY-h3w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_MFmAF07BBrc.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_F14sWKylN6c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_qUphd3XFpW0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa6c_2023_06_30/tile_QtvRPAbpGLo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2023_06_30/tile_6RNE_2tn9Z0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2023_06_30/tile_cI_89vaQokA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2023_06_30/tile_5sFxLBA_Iqk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2023_06_30/tile_SrA3hNKGZ64.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2023_06_30/tile_Dd4GpvS8Ge0.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2023_06_30/tile_YNgafwyhidk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2023_06_30/tile_t8LbcNrkWrE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2023_06_30/tile_SCFPnVfUql8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa5c_2023_06_30/tile_xVbV88yWUhE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2023_06_30/tile_I1OMvjAHoc8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2023_06_30/tile_MVyILXNgnas.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2023_06_30/tile_dPY0-2oQmtU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2023_06_30/tile_LBRA9z5hNF4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2023_06_30/tile_XacG0jFDyH8.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2023_06_30/tile_5WXILtuOQbo.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2023_06_30/tile_uedmM83ZqRs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2023_06_30/tile_H42ZeH4gFtI.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2023_06_30/tile_4PJBpAUrAqw.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2023_06_30/tile_y4YR_QsaTcQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2023_06_30/tile_72OrTmz8LwY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2023_06_30/tile_45CwXin5MhE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2023_06_30/tile_rxgV4aC8GWg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2023_06_30/tile_hn2htg6Y1KQ.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2023_06_30/tile_R6kRjlEAS2Y.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8fa4c_2023_06_30/tile_PZ7IGn7F_pE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_878QAjksX9k.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_5AGXayBwZCM.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_95lAfaJ_1o4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_RnIje33rRLY.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_iFHrjLE08Vg.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_xYnut80Y-zE.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_HdPOmocD19w.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_UlfQI_P7vco.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_G2nePRpiK6Q.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_jVMuoz3FmBA.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile__lHwd88WlAk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_uA6JC01J-II.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_n6WL2zzxprU.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_e7OyCSqpV-c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_MeDOXFSvFNs.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_uptv8iGJBi4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_PiL6Joun_-M.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_j1y0l7fP-Sk.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_NYfDvlcPv0c.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_KhF1V8P1Z20.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_wcCkxNyBXr4.tif',
	'gs://open-buildings-temporal-data/v1/geotiffs/8f0a4_2023_06_30/tile_MJ1U3NpVqR8.tif',
]

# Carpeta de destino en Google Drive
output_folder = '/content/drive/MyDrive/GeoTIFF_BUILDINGS'
os.makedirs(output_folder, exist_ok=True)

# Función para descargar imágenes
for uri in geotiff_uris:
    try:
        # Cargar la imagen desde el URI
        image = ee.Image.loadGeoTIFF(uri)

        # Definir el nombre del archivo
        filename = uri.split('/')[-1]
        local_path = os.path.join(output_folder, filename)

        # Exportar la imagen como GeoTIFF
        print(f"Descargando: {filename}")
        ee.batch.Export.image.toDrive(
            image=image,
            description=filename,
            folder='GeoTIFFs',
            fileNamePrefix=filename.split('.')[0],
            scale=10,
            region=image.geometry().bounds()
        ).start()
    except Exception as e:
        print(f"Error descargando {uri}: {e}")

print(f"Las imágenes han sido exportadas a Google Drive en la carpeta {output_folder}.")